In [ ]:
## running code

# robotuser@kuorobot02 carla_0.9.12 → python3 carla_lane_env.py 
# robotuser@kuorobot02 carla_0.9.12 → python3 carla_rl_training.py 

#carla_0.9.12 → python3 carla_evaluation.py  --model carla_dqn_arrow_best.pth --episodes 3
#robotuser@kuorobot02 carla_0.9.12 → python3 carla_evaluation.py  --model carla_dqn_smooth_final.pth --episodes 5

 


# carla_lane.py


<!-- robotuser@kuorobot02 carla_0.9.12 → python3 carla_evaluation.py  --model carla_dqn_arrow_best.pth --episodes 3

robotuser@kuorobot02 carla_0.9.12 → python3 carla_evaluation.py  --model carla_dqn_smooth_final.pth --episodes 5

 robotuser@kuorobot02 carla_0.9.12 → python3 carla_rl_training.py 

 robotuser@kuorobot02 carla_0.9.12 → python3 carla_evaluation.py  --model carla_dqn_agent_final.pth --duration 120 --speed 0.3

robotuser@kuorobot02 carla_0.9.12 → python3 carla_lane_env.py  -->


In [ ]:
"""
CARLA RL Training using DQN - WITH LANE ARROW FOLLOWING
Focus: Smooth driving + Following lane markings (left/right arrows)

Key Features:
1. Detects lane change arrows in current lane
2. Rewards following the arrow direction
3. Smooth lane changes when arrows detected
4. Enhanced state representation with lane change info
"""

import carla
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import time

# ======================= CONFIG =======================
EPISODES = 300
MAX_STEPS = 500
MEMORY_SIZE = 50000
BATCH_SIZE = 128
GAMMA = 0.99
EPSILON_START = 1.0
EPSILON_END = 0.05
EPSILON_DECAY = 0.995
LEARNING_RATE = 1e-4
TARGET_UPDATE = 10

# IMPROVED DISCRETE ACTIONS (smoother, more gradual)
ACTIONS = {
    0: (0.5, 0.0),    # Straight forward
    1: (0.5, -0.15),  # Gentle left
    2: (0.5, 0.15),   # Gentle right
    3: (0.4, 0.0),    # Slow forward
    4: (0.4, -0.15),  # Slow + gentle left
    5: (0.4, 0.15),   # Slow + gentle right
    6: (0.0, 0.0),    # Brake/Stop
    7: (0.3, -0.3),   # Sharp left (for lane changes)
    8: (0.3, 0.3),    # Sharp right (for lane changes)
}

# IMPROVED REWARDS
REWARD_DISTANCE = 1.0          
REWARD_SPEED = 1.5             
REWARD_LANE_CENTER = 5.0       
REWARD_HEADING = 3.0           
REWARD_SMOOTHNESS = 2.0        
REWARD_COLLISION = -200.0      
REWARD_OFF_ROAD = -100.0       
REWARD_LANE_INVASION = -20.0   
REWARD_JERKY = -5.0            

# NEW: Lane arrow following rewards
REWARD_FOLLOW_ARROW = 10.0      # Strong reward for following lane arrow
REWARD_WRONG_LANE_CHANGE = -15.0  # Penalty for changing lane wrong direction
REWARD_SUCCESSFUL_LANE_CHANGE = 20.0  # Bonus for completing lane change correctly

TARGET_SPEED = 8.0

# ======================= DQN NETWORK =======================
class DQN(nn.Module):
    def __init__(self, state_dim=18, action_dim=9):  # Increased state_dim to 18
        super(DQN, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )
    
    def forward(self, x):
        return self.network(x)

# ======================= REPLAY MEMORY =======================
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

# ======================= IMPROVED CARLA ENVIRONMENT =======================
class ImprovedCarlaEnv:
    def __init__(self, town='Town04', port=2000):
        self.client = carla.Client('localhost', port)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world(town)
        
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()
        self.vehicle = None
        self.sensors = []
        
        self.collision_hist = []
        self.lane_invasion_hist = []
        self.spawn_points = self.map.get_spawn_points()
        
        self.last_location = None
        self.distance_traveled = 0.0
        self.steps = 0
        
        # For smoothness tracking
        self.last_steer = 0.0
        self.last_throttle = 0.5
        self.steer_history = deque(maxlen=5)
        
        # NEW: Lane change tracking
        self.current_lane_id = None
        self.target_lane_from_arrow = None  # "left", "right", or None
        self.lane_change_in_progress = False
        self.lane_change_start_time = 0
        self.successful_lane_changes = 0
        
        print("Improved Environment initialized with Lane Arrow Following!")
    
    def reset(self):
        """Reset environment"""
        self._cleanup()
        
        vehicle_bp = self.blueprint_library.filter('vehicle.tesla.model3')[0]
        spawn_point = random.choice(self.spawn_points[:30])
        
        try:
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        except:
            spawn_point = random.choice(self.spawn_points)
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        
        # Collision sensor
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        collision_sensor.listen(lambda e: self.collision_hist.append(e))
        self.sensors.append(collision_sensor)
        
        # Lane invasion sensor
        lane_bp = self.blueprint_library.find('sensor.other.lane_invasion')
        lane_sensor = self.world.spawn_actor(
            lane_bp, carla.Transform(), attach_to=self.vehicle
        )
        lane_sensor.listen(lambda e: self.lane_invasion_hist.append(e))
        self.sensors.append(lane_sensor)
        
        # Reset tracking
        self.collision_hist.clear()
        self.lane_invasion_hist.clear()
        self.last_location = self.vehicle.get_location()
        self.distance_traveled = 0.0
        self.steps = 0
        self.last_steer = 0.0
        self.last_throttle = 0.5
        self.steer_history.clear()
        
        # Reset lane change tracking
        self.current_lane_id = None
        self.target_lane_from_arrow = None
        self.lane_change_in_progress = False
        self.lane_change_start_time = 0
        self.successful_lane_changes = 0
        
        # Warmup
        for _ in range(10):
            self.world.tick()
        
        # Initialize lane ID
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint:
            self.current_lane_id = waypoint.lane_id
        
        return self._get_state()
    
    def _detect_lane_arrow(self, waypoint):
        """
        Detect if current lane has left/right arrow marking
        Returns: "left", "right", "both", or None
        """
        if waypoint is None:
            return None
        
        # Get lane change permission
        lane_change = waypoint.lane_change
        
        # CARLA LaneChange enum values:
        # - NONE: No lane change allowed
        # - Right: Can change to right
        # - Left: Can change to left  
        # - Both: Can change to both sides
        
        if lane_change == carla.LaneChange.Left:
            return "left"
        elif lane_change == carla.LaneChange.Right:
            return "right"
        elif lane_change == carla.LaneChange.Both:
            return "both"
        else:
            return None
    
    def _check_lane_availability(self, waypoint, direction):
        """Check if lane change to direction is safe and valid"""
        if waypoint is None or direction is None:
            return False
        
        try:
            if direction == "left":
                target_wp = waypoint.get_left_lane()
            elif direction == "right":
                target_wp = waypoint.get_right_lane()
            else:
                return False
            
            if target_wp is None:
                return False
            
            # Must be driving lane
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            # Must be same direction
            if (waypoint.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            return True
            
        except Exception:
            return False
    
    def _get_state(self):
        """Enhanced state representation WITH lane arrow info"""
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        waypoint = self.map.get_waypoint(
            transform.location, 
            project_to_road=True, 
            lane_type=carla.LaneType.Driving
        )
        
        if waypoint is None:
            return np.zeros(18, dtype=np.float32)
        
        # Lane offset (cross-track error)
        lane_center = waypoint.transform.location
        offset_vec = transform.location - lane_center
        right_vec = transform.get_right_vector()
        lane_offset = offset_vec.x * right_vec.x + offset_vec.y * right_vec.y
        
        # Heading error
        lane_yaw = waypoint.transform.rotation.yaw
        vehicle_yaw = transform.rotation.yaw
        heading_error = (vehicle_yaw - lane_yaw + 180) % 360 - 180
        heading_error_rad = np.radians(heading_error)
        
        # Look-ahead waypoint
        next_waypoints = waypoint.next(5.0)
        if next_waypoints:
            next_wp = next_waypoints[0]
            wp_vec = next_wp.transform.location - transform.location
            forward_vec = transform.get_forward_vector()
            angle_to_wp = np.arctan2(
                wp_vec.y * forward_vec.x - wp_vec.x * forward_vec.y,
                wp_vec.x * forward_vec.x + wp_vec.y * forward_vec.y
            )
        else:
            angle_to_wp = 0.0
        
        # Road curvature
        curvature = 0.0
        if next_waypoints:
            next_wp = next_waypoints[0]
            curvature_yaw = next_wp.transform.rotation.yaw - lane_yaw
            curvature = np.sin(np.radians(curvature_yaw))
        
        # Traffic light state
        tl_state = 0.0
        if self.vehicle.is_at_traffic_light():
            tl = self.vehicle.get_traffic_light()
            if tl:
                state = tl.get_state()
                if state == carla.TrafficLightState.Green:
                    tl_state = 0.33
                elif state == carla.TrafficLightState.Yellow:
                    tl_state = 0.66
                elif state == carla.TrafficLightState.Red:
                    tl_state = 1.0
        
        # Steering smoothness
        steer_variance = np.std(list(self.steer_history)) if len(self.steer_history) > 2 else 0.0
        
        # NEW: Lane arrow detection
        lane_arrow = self._detect_lane_arrow(waypoint)
        
        # Encode lane arrow as three binary flags
        arrow_left = 1.0 if lane_arrow in ["left", "both"] else 0.0
        arrow_right = 1.0 if lane_arrow in ["right", "both"] else 0.0
        arrow_straight = 1.0 if lane_arrow is None else 0.0
        
        # Check if lanes are actually available
        can_change_left = 1.0 if self._check_lane_availability(waypoint, "left") else 0.0
        can_change_right = 1.0 if self._check_lane_availability(waypoint, "right") else 0.0
        
        # Update target from arrow (only if not already in lane change)
        if not self.lane_change_in_progress:
            if arrow_left and can_change_left:
                self.target_lane_from_arrow = "left"
            elif arrow_right and can_change_right:
                self.target_lane_from_arrow = "right"
            else:
                self.target_lane_from_arrow = None
        
        state = np.array([
            speed / 30.0,                          # 0: Normalized speed
            lane_offset / 3.5,                     # 1: Lane offset
            np.sin(heading_error_rad),             # 2: Heading error (sin)
            np.cos(heading_error_rad),             # 3: Heading error (cos)
            np.sin(angle_to_wp),                   # 4: Angle to waypoint (sin)
            np.cos(angle_to_wp),                   # 5: Angle to waypoint (cos)
            curvature,                             # 6: Road curvature
            velocity.x / 30.0,                     # 7: Velocity X
            velocity.y / 30.0,                     # 8: Velocity Y
            float(waypoint.is_junction),           # 9: Junction flag
            tl_state,                              # 10: Traffic light state
            self.last_steer,                       # 11: Previous steering
            steer_variance,                        # 12: Steering smoothness
            arrow_left,                            # 13: Lane arrow left
            arrow_right,                           # 14: Lane arrow right
            arrow_straight,                        # 15: No arrow (straight)
            can_change_left,                       # 16: Left lane available
            can_change_right,                      # 17: Right lane available
        ], dtype=np.float32)
        
        return state
    
    def step(self, action_idx):
        """Execute action with smoothing"""
        throttle_raw, steer_raw = ACTIONS[action_idx]
        
        # Apply smoothing
        alpha = 0.3
        throttle = alpha * throttle_raw + (1 - alpha) * self.last_throttle
        steer = alpha * steer_raw + (1 - alpha) * self.last_steer
        
        # Limit steering rate of change
        max_steer_change = 0.1
        steer_change = np.clip(steer - self.last_steer, -max_steer_change, max_steer_change)
        steer = self.last_steer + steer_change
        
        # Store for next iteration
        self.last_steer = steer
        self.last_throttle = throttle
        self.steer_history.append(steer)
        
        # Apply control
        control = carla.VehicleControl(
            throttle=float(throttle),
            steer=float(steer),
            brake=0.0 if throttle > 0 else 0.5
        )
        
        self.vehicle.apply_control(control)
        self.world.tick()
        self.steps += 1
        
        # Track lane changes
        prev_invasions = len(self.lane_invasion_hist)
        prev_lane_id = self.current_lane_id
        
        # Update distance
        current_location = self.vehicle.get_location()
        distance_step = 0.0
        if self.last_location:
            distance_step = current_location.distance(self.last_location)
            self.distance_traveled += distance_step
        self.last_location = current_location
        
        # Update current lane
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint:
            self.current_lane_id = waypoint.lane_id
        
        # Detect lane change completion
        lane_changed = False
        lane_change_direction = None
        if prev_lane_id is not None and self.current_lane_id != prev_lane_id:
            lane_changed = True
            # Determine direction (left = positive lane_id change, right = negative)
            if self.current_lane_id > prev_lane_id:
                lane_change_direction = "left"
            else:
                lane_change_direction = "right"
            
            # Check if it matches the arrow
            if lane_change_direction == self.target_lane_from_arrow:
                self.successful_lane_changes += 1
            
            # Reset lane change tracking
            self.lane_change_in_progress = False
            self.target_lane_from_arrow = None
        
        # Detect lane change initiation (significant lateral velocity)
        if not self.lane_change_in_progress and abs(steer_raw) > 0.2:
            self.lane_change_in_progress = True
            self.lane_change_start_time = self.steps
        
        # Get new state
        next_state = self._get_state()
        
        # Calculate reward
        reward = self._calculate_reward(
            distance_step, 
            prev_invasions, 
            steer_change,
            lane_changed,
            lane_change_direction
        )
        
        # Check done
        done = self._is_done()
        
        return next_state, reward, done
    
    def _calculate_reward(self, distance_step, prev_invasions, steer_change, 
                         lane_changed, lane_change_direction):
        """Improved reward with lane arrow following"""
        reward = 0.0
        
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        # 1. Forward progress
        reward += REWARD_DISTANCE * distance_step
        
        # 2. Speed reward
        speed_error = abs(speed - TARGET_SPEED)
        speed_reward = REWARD_SPEED * np.exp(-speed_error / 5.0)
        reward += speed_reward
        
        # 3. Lane keeping reward
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        if waypoint:
            lane_center = waypoint.transform.location
            offset_vec = transform.location - lane_center
            right_vec = transform.get_right_vector()
            lane_offset = abs(offset_vec.x * right_vec.x + offset_vec.y * right_vec.y)
            
            lane_reward = REWARD_LANE_CENTER * np.exp(-lane_offset * 3.0)
            reward += lane_reward
            
            # Heading alignment
            lane_yaw = waypoint.transform.rotation.yaw
            vehicle_yaw = transform.rotation.yaw
            heading_error = abs((vehicle_yaw - lane_yaw + 180) % 360 - 180)
            heading_reward = REWARD_HEADING * np.exp(-heading_error / 30.0)
            reward += heading_reward
        else:
            reward += REWARD_OFF_ROAD
        
        # 4. Smoothness reward
        steer_jerk = abs(steer_change)
        if steer_jerk > 0.05:
            reward += REWARD_JERKY * steer_jerk
        else:
            reward += REWARD_SMOOTHNESS * (0.05 - steer_jerk)
        
        # 5. Lane invasion penalty
        if len(self.lane_invasion_hist) > prev_invasions:
            reward += REWARD_LANE_INVASION
        
        # 6. Collision penalty
        if len(self.collision_hist) > 0:
            reward += REWARD_COLLISION
        
        # 7. Bonus for good driving
        if waypoint and lane_offset < 0.5 and speed > 5.0:
            reward += 1.0
        
        # NEW: 8. Lane arrow following rewards
        if lane_changed:
            if lane_change_direction == self.target_lane_from_arrow:
                # Correct lane change following arrow!
                reward += REWARD_SUCCESSFUL_LANE_CHANGE
                print(f"  ✓ Successful {lane_change_direction} lane change following arrow!")
            else:
                # Wrong direction lane change
                reward += REWARD_WRONG_LANE_CHANGE
        
        # NEW: 9. Continuous reward for moving toward arrow direction
        if self.target_lane_from_arrow is not None and waypoint:
            # Small reward for steering in correct direction when arrow present
            if self.target_lane_from_arrow == "left" and self.last_steer < -0.05:
                reward += REWARD_FOLLOW_ARROW * 0.1  # Small continuous reward
            elif self.target_lane_from_arrow == "right" and self.last_steer > 0.05:
                reward += REWARD_FOLLOW_ARROW * 0.1
        
        return reward
    
    def _is_done(self):
        """Check termination"""
        if len(self.collision_hist) > 0:
            return True
        
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        if waypoint is None:
            return True
        
        if transform.location.distance(waypoint.transform.location) > 4.0:
            return True
        
        if self.steps >= MAX_STEPS:
            return True
        
        return False
    
    def _cleanup(self):
        """Cleanup"""
        if self.vehicle:
            self.vehicle.destroy()
        for sensor in self.sensors:
            if sensor.is_alive:
                sensor.destroy()
        self.sensors.clear()
        self.vehicle = None
    
    def close(self):
        """Close environment"""
        self._cleanup()
        settings = self.world.get_settings()
        settings.synchronous_mode = False
        self.world.apply_settings(settings)

# ======================= DQN AGENT =======================
class DQNAgent:
    def __init__(self, state_dim=18, action_dim=9):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.policy_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=LEARNING_RATE)
        self.memory = ReplayMemory(MEMORY_SIZE)
        
        self.epsilon = EPSILON_START
        self.action_dim = action_dim
    
    def select_action(self, state):
        """Epsilon-greedy action selection"""
        if random.random() < self.epsilon:
            return random.randrange(self.action_dim)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.policy_net(state_tensor)
            return q_values.argmax().item()
    
    def update(self):
        """Update network"""
        if len(self.memory) < BATCH_SIZE:
            return 0.0
        
        transitions = self.memory.sample(BATCH_SIZE)
        batch = list(zip(*transitions))
        
        state_batch = torch.FloatTensor(np.array(batch[0])).to(self.device)
        action_batch = torch.LongTensor(batch[1]).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch[2]).to(self.device)
        next_state_batch = torch.FloatTensor(np.array(batch[3])).to(self.device)
        done_batch = torch.FloatTensor(batch[4]).to(self.device)
        
        current_q = self.policy_net(state_batch).gather(1, action_batch)
        
        with torch.no_grad():
            next_actions = self.policy_net(next_state_batch).argmax(1, keepdim=True)
            next_q = self.target_net(next_state_batch).gather(1, next_actions).squeeze()
            target_q = reward_batch + (1 - done_batch) * GAMMA * next_q
        
        loss = nn.SmoothL1Loss()(current_q.squeeze(), target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def update_target_network(self):
        """Update target network"""
        self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def decay_epsilon(self):
        """Decay exploration"""
        self.epsilon = max(EPSILON_END, self.epsilon * EPSILON_DECAY)

# ======================= TRAINING =======================
def train():
    print("="*70)
    print("CARLA DQN TRAINING - SMOOTH DRIVING + LANE ARROW FOLLOWING")
    print("="*70)
    
    env = ImprovedCarlaEnv()
    agent = DQNAgent(state_dim=18, action_dim=9)
    
    episode_rewards = []
    episode_distances = []
    episode_lengths = []
    episode_lane_invasions = []
    episode_lane_changes = []
    
    print("\nStarting training...")
    print("Focus: Smooth control + Lane keeping + Following lane arrows\n")
    start_time = time.time()
    
    best_avg_distance = 0.0
    
    for episode in range(EPISODES):
        state = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)
            
            agent.memory.push(state, action, reward, next_state, float(done))
            loss = agent.update()
            
            episode_reward += reward
            state = next_state
        
        if (episode + 1) % TARGET_UPDATE == 0:
            agent.update_target_network()
        
        agent.decay_epsilon()
        
        # Track metrics
        episode_rewards.append(episode_reward)
        episode_distances.append(env.distance_traveled)
        episode_lengths.append(env.steps)
        episode_lane_invasions.append(len(env.lane_invasion_hist))
        episode_lane_changes.append(env.successful_lane_changes)
        
        # Logging
        if (episode + 1) % 10 == 0:
            avg_reward = np.mean(episode_rewards[-10:])
            avg_distance = np.mean(episode_distances[-10:])
            avg_length = np.mean(episode_lengths[-10:])
            avg_invasions = np.mean(episode_lane_invasions[-10:])
            avg_changes = np.mean(episode_lane_changes[-10:])
            elapsed = (time.time() - start_time) / 60
            
            print(f"Ep {episode+1:3d} | "
                  f"R: {episode_reward:7.1f} | "
                  f"D: {env.distance_traveled:6.1f}m | "
                  f"L: {env.steps:3d} | "
                  f"LI: {len(env.lane_invasion_hist):2d} | "
                  f"LC: {env.successful_lane_changes} | "
                  f"ε: {agent.epsilon:.3f} | "
                  f"Avg10: R={avg_reward:6.1f}, D={avg_distance:5.1f}m, LC={avg_changes:.1f} | "
                  f"T: {elapsed:.1f}min")
            
            if avg_distance > best_avg_distance:
                best_avg_distance = avg_distance
                torch.save(agent.policy_net.state_dict(), 'carla_dqn_arrow_best.pth')
                print(f"  ✓ New best model saved! Avg distance: {avg_distance:.1f}m")
        
        if (episode + 1) % 50 == 0:
            torch.save(agent.policy_net.state_dict(), f'dqn_arrow_ep{episode+1}.pth')
            print(f"  ✓ Checkpoint saved")
    
    torch.save(agent.policy_net.state_dict(), 'carla_dqn_arrow_final.pth')
    print(f"\n✓ Training completed in {(time.time() - start_time) / 60:.1f} minutes")
    print("✓ Final model saved: carla_dqn_arrow_final.pth")
    
    # Summary
    print("\n" + "="*70)
    print("TRAINING SUMMARY")
    print("="*70)
    print(f"Best avg distance (10 ep): {best_avg_distance:.1f}m")
    print(f"Final avg reward (last 20): {np.mean(episode_rewards[-20:]):.1f}")
    print(f"Final avg distance (last 20): {np.mean(episode_distances[-20:]):.1f}m")
    print(f"Final avg lane invasions (last 20): {np.mean(episode_lane_invasions[-20:]):.1f}")
    print(f"Final avg successful lane changes (last 20): {np.mean(episode_lane_changes[-20:]):.1f}")
    print("="*70)
    
    env.close()

if __name__ == "__main__":
    train()

# carla_rl_training.py

## dqn_arrow_best.pth

In [ ]:
"""
CARLA RL Training using DQN - WITH LANE ARROW FOLLOWING
Focus: Smooth driving + Following lane markings (left/right arrows)

Key Features:
1. Detects lane change arrows in current lane
2. Rewards following the arrow direction
3. Smooth lane changes when arrows detected
4. Enhanced state representation with lane change info
"""

import carla
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import time

# ======================= CONFIG =======================
EPISODES = 300
MAX_STEPS = 500
MEMORY_SIZE = 50000
BATCH_SIZE = 128
GAMMA = 0.99
EPSILON_START = 1.0
EPSILON_END = 0.05
EPSILON_DECAY = 0.995
LEARNING_RATE = 1e-4
TARGET_UPDATE = 10

# IMPROVED DISCRETE ACTIONS (smoother, more gradual)
ACTIONS = {
    0: (0.5, 0.0),    # Straight forward
    1: (0.5, -0.15),  # Gentle left
    2: (0.5, 0.15),   # Gentle right
    3: (0.4, 0.0),    # Slow forward
    4: (0.4, -0.15),  # Slow + gentle left
    5: (0.4, 0.15),   # Slow + gentle right
    6: (0.0, 0.0),    # Brake/Stop
    7: (0.3, -0.3),   # Sharp left (for lane changes)
    8: (0.3, 0.3),    # Sharp right (for lane changes)
}

# IMPROVED REWARDS
REWARD_DISTANCE = 1.0          
REWARD_SPEED = 1.5             
REWARD_LANE_CENTER = 5.0       
REWARD_HEADING = 3.0           
REWARD_SMOOTHNESS = 2.0        
REWARD_COLLISION = -200.0      
REWARD_OFF_ROAD = -100.0       
REWARD_LANE_INVASION = -20.0   
REWARD_JERKY = -5.0            

# NEW: Lane arrow following rewards
REWARD_FOLLOW_ARROW = 10.0      # Strong reward for following lane arrow
REWARD_WRONG_LANE_CHANGE = -15.0  # Penalty for changing lane wrong direction
REWARD_SUCCESSFUL_LANE_CHANGE = 20.0  # Bonus for completing lane change correctly

TARGET_SPEED = 8.0

# ======================= DQN NETWORK =======================
class DQN(nn.Module):
    def __init__(self, state_dim=18, action_dim=9):  # Increased state_dim to 18
        super(DQN, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )
    
    def forward(self, x):
        return self.network(x)

# ======================= REPLAY MEMORY =======================
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

# ======================= IMPROVED CARLA ENVIRONMENT =======================
class ImprovedCarlaEnv:
    def __init__(self, town='Town04', port=2000):
        self.client = carla.Client('localhost', port)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world(town)
        
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()
        self.vehicle = None
        self.sensors = []
        
        self.collision_hist = []
        self.lane_invasion_hist = []
        self.spawn_points = self.map.get_spawn_points()
        
        self.last_location = None
        self.distance_traveled = 0.0
        self.steps = 0
        
        # For smoothness tracking
        self.last_steer = 0.0
        self.last_throttle = 0.5
        self.steer_history = deque(maxlen=5)
        
        # NEW: Lane change tracking
        self.current_lane_id = None
        self.target_lane_from_arrow = None  # "left", "right", or None
        self.lane_change_in_progress = False
        self.lane_change_start_time = 0
        self.successful_lane_changes = 0
        
        print("Improved Environment initialized with Lane Arrow Following!")
    
    def reset(self):
        """Reset environment"""
        self._cleanup()
        
        vehicle_bp = self.blueprint_library.filter('vehicle.tesla.model3')[0]
        spawn_point = random.choice(self.spawn_points[:30])
        
        try:
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        except:
            spawn_point = random.choice(self.spawn_points)
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        
        # Collision sensor
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        collision_sensor.listen(lambda e: self.collision_hist.append(e))
        self.sensors.append(collision_sensor)
        
        # Lane invasion sensor
        lane_bp = self.blueprint_library.find('sensor.other.lane_invasion')
        lane_sensor = self.world.spawn_actor(
            lane_bp, carla.Transform(), attach_to=self.vehicle
        )
        lane_sensor.listen(lambda e: self.lane_invasion_hist.append(e))
        self.sensors.append(lane_sensor)
        
        # Reset tracking
        self.collision_hist.clear()
        self.lane_invasion_hist.clear()
        self.last_location = self.vehicle.get_location()
        self.distance_traveled = 0.0
        self.steps = 0
        self.last_steer = 0.0
        self.last_throttle = 0.5
        self.steer_history.clear()
        
        # Reset lane change tracking
        self.current_lane_id = None
        self.target_lane_from_arrow = None
        self.lane_change_in_progress = False
        self.lane_change_start_time = 0
        self.successful_lane_changes = 0
        
        # Warmup
        for _ in range(10):
            self.world.tick()
        
        # Initialize lane ID
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint:
            self.current_lane_id = waypoint.lane_id
        
        return self._get_state()
    
    def _detect_lane_arrow(self, waypoint):
        """
        Detect if current lane has left/right arrow marking
        Returns: "left", "right", "both", or None
        """
        if waypoint is None:
            return None
        
        # Get lane change permission
        lane_change = waypoint.lane_change
        
        # CARLA LaneChange enum values:
        # - NONE: No lane change allowed
        # - Right: Can change to right
        # - Left: Can change to left  
        # - Both: Can change to both sides
        
        if lane_change == carla.LaneChange.Left:
            return "left"
        elif lane_change == carla.LaneChange.Right:
            return "right"
        elif lane_change == carla.LaneChange.Both:
            return "both"
        else:
            return None
    
    def _check_lane_availability(self, waypoint, direction):
        """Check if lane change to direction is safe and valid"""
        if waypoint is None or direction is None:
            return False
        
        try:
            if direction == "left":
                target_wp = waypoint.get_left_lane()
            elif direction == "right":
                target_wp = waypoint.get_right_lane()
            else:
                return False
            
            if target_wp is None:
                return False
            
            # Must be driving lane
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            # Must be same direction
            if (waypoint.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            return True
            
        except Exception:
            return False
    
    def _get_state(self):
        """Enhanced state representation WITH lane arrow info"""
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        waypoint = self.map.get_waypoint(
            transform.location, 
            project_to_road=True, 
            lane_type=carla.LaneType.Driving
        )
        
        if waypoint is None:
            return np.zeros(18, dtype=np.float32)
        
        # Lane offset (cross-track error)
        lane_center = waypoint.transform.location
        offset_vec = transform.location - lane_center
        right_vec = transform.get_right_vector()
        lane_offset = offset_vec.x * right_vec.x + offset_vec.y * right_vec.y
        
        # Heading error
        lane_yaw = waypoint.transform.rotation.yaw
        vehicle_yaw = transform.rotation.yaw
        heading_error = (vehicle_yaw - lane_yaw + 180) % 360 - 180
        heading_error_rad = np.radians(heading_error)
        
        # Look-ahead waypoint
        next_waypoints = waypoint.next(5.0)
        if next_waypoints:
            next_wp = next_waypoints[0]
            wp_vec = next_wp.transform.location - transform.location
            forward_vec = transform.get_forward_vector()
            angle_to_wp = np.arctan2(
                wp_vec.y * forward_vec.x - wp_vec.x * forward_vec.y,
                wp_vec.x * forward_vec.x + wp_vec.y * forward_vec.y
            )
        else:
            angle_to_wp = 0.0
        
        # Road curvature
        curvature = 0.0
        if next_waypoints:
            next_wp = next_waypoints[0]
            curvature_yaw = next_wp.transform.rotation.yaw - lane_yaw
            curvature = np.sin(np.radians(curvature_yaw))
        
        # Traffic light state
        tl_state = 0.0
        if self.vehicle.is_at_traffic_light():
            tl = self.vehicle.get_traffic_light()
            if tl:
                state = tl.get_state()
                if state == carla.TrafficLightState.Green:
                    tl_state = 0.33
                elif state == carla.TrafficLightState.Yellow:
                    tl_state = 0.66
                elif state == carla.TrafficLightState.Red:
                    tl_state = 1.0
        
        # Steering smoothness
        steer_variance = np.std(list(self.steer_history)) if len(self.steer_history) > 2 else 0.0
        
        # NEW: Lane arrow detection
        lane_arrow = self._detect_lane_arrow(waypoint)
        
        # Encode lane arrow as three binary flags
        arrow_left = 1.0 if lane_arrow in ["left", "both"] else 0.0
        arrow_right = 1.0 if lane_arrow in ["right", "both"] else 0.0
        arrow_straight = 1.0 if lane_arrow is None else 0.0
        
        # Check if lanes are actually available
        can_change_left = 1.0 if self._check_lane_availability(waypoint, "left") else 0.0
        can_change_right = 1.0 if self._check_lane_availability(waypoint, "right") else 0.0
        
        # Update target from arrow (only if not already in lane change)
        if not self.lane_change_in_progress:
            if arrow_left and can_change_left:
                self.target_lane_from_arrow = "left"
            elif arrow_right and can_change_right:
                self.target_lane_from_arrow = "right"
            else:
                self.target_lane_from_arrow = None
        
        state = np.array([
            speed / 30.0,                          # 0: Normalized speed
            lane_offset / 3.5,                     # 1: Lane offset
            np.sin(heading_error_rad),             # 2: Heading error (sin)
            np.cos(heading_error_rad),             # 3: Heading error (cos)
            np.sin(angle_to_wp),                   # 4: Angle to waypoint (sin)
            np.cos(angle_to_wp),                   # 5: Angle to waypoint (cos)
            curvature,                             # 6: Road curvature
            velocity.x / 30.0,                     # 7: Velocity X
            velocity.y / 30.0,                     # 8: Velocity Y
            float(waypoint.is_junction),           # 9: Junction flag
            tl_state,                              # 10: Traffic light state
            self.last_steer,                       # 11: Previous steering
            steer_variance,                        # 12: Steering smoothness
            arrow_left,                            # 13: Lane arrow left
            arrow_right,                           # 14: Lane arrow right
            arrow_straight,                        # 15: No arrow (straight)
            can_change_left,                       # 16: Left lane available
            can_change_right,                      # 17: Right lane available
        ], dtype=np.float32)
        
        return state
    
    def step(self, action_idx):
        """Execute action with smoothing"""
        throttle_raw, steer_raw = ACTIONS[action_idx]
        
        # Apply smoothing
        alpha = 0.3
        throttle = alpha * throttle_raw + (1 - alpha) * self.last_throttle
        steer = alpha * steer_raw + (1 - alpha) * self.last_steer
        
        # Limit steering rate of change
        max_steer_change = 0.1
        steer_change = np.clip(steer - self.last_steer, -max_steer_change, max_steer_change)
        steer = self.last_steer + steer_change
        
        # Store for next iteration
        self.last_steer = steer
        self.last_throttle = throttle
        self.steer_history.append(steer)
        
        # Apply control
        control = carla.VehicleControl(
            throttle=float(throttle),
            steer=float(steer),
            brake=0.0 if throttle > 0 else 0.5
        )
        
        self.vehicle.apply_control(control)
        self.world.tick()
        self.steps += 1
        
        # Track lane changes
        prev_invasions = len(self.lane_invasion_hist)
        prev_lane_id = self.current_lane_id
        
        # Update distance
        current_location = self.vehicle.get_location()
        distance_step = 0.0
        if self.last_location:
            distance_step = current_location.distance(self.last_location)
            self.distance_traveled += distance_step
        self.last_location = current_location
        
        # Update current lane
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint:
            self.current_lane_id = waypoint.lane_id
        
        # Detect lane change completion
        lane_changed = False
        lane_change_direction = None
        if prev_lane_id is not None and self.current_lane_id != prev_lane_id:
            lane_changed = True
            # Determine direction (left = positive lane_id change, right = negative)
            if self.current_lane_id > prev_lane_id:
                lane_change_direction = "left"
            else:
                lane_change_direction = "right"
            
            # Check if it matches the arrow
            if lane_change_direction == self.target_lane_from_arrow:
                self.successful_lane_changes += 1
            
            # Reset lane change tracking
            self.lane_change_in_progress = False
            self.target_lane_from_arrow = None
        
        # Detect lane change initiation (significant lateral velocity)
        if not self.lane_change_in_progress and abs(steer_raw) > 0.2:
            self.lane_change_in_progress = True
            self.lane_change_start_time = self.steps
        
        # Get new state
        next_state = self._get_state()
        
        # Calculate reward
        reward = self._calculate_reward(
            distance_step, 
            prev_invasions, 
            steer_change,
            lane_changed,
            lane_change_direction
        )
        
        # Check done
        done = self._is_done()
        
        return next_state, reward, done
    
    def _calculate_reward(self, distance_step, prev_invasions, steer_change, 
                         lane_changed, lane_change_direction):
        """Improved reward with lane arrow following"""
        reward = 0.0
        
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        # 1. Forward progress
        reward += REWARD_DISTANCE * distance_step
        
        # 2. Speed reward
        speed_error = abs(speed - TARGET_SPEED)
        speed_reward = REWARD_SPEED * np.exp(-speed_error / 5.0)
        reward += speed_reward
        
        # 3. Lane keeping reward
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        if waypoint:
            lane_center = waypoint.transform.location
            offset_vec = transform.location - lane_center
            right_vec = transform.get_right_vector()
            lane_offset = abs(offset_vec.x * right_vec.x + offset_vec.y * right_vec.y)
            
            lane_reward = REWARD_LANE_CENTER * np.exp(-lane_offset * 3.0)
            reward += lane_reward
            
            # Heading alignment
            lane_yaw = waypoint.transform.rotation.yaw
            vehicle_yaw = transform.rotation.yaw
            heading_error = abs((vehicle_yaw - lane_yaw + 180) % 360 - 180)
            heading_reward = REWARD_HEADING * np.exp(-heading_error / 30.0)
            reward += heading_reward
        else:
            reward += REWARD_OFF_ROAD
        
        # 4. Smoothness reward
        steer_jerk = abs(steer_change)
        if steer_jerk > 0.05:
            reward += REWARD_JERKY * steer_jerk
        else:
            reward += REWARD_SMOOTHNESS * (0.05 - steer_jerk)
        
        # 5. Lane invasion penalty
        if len(self.lane_invasion_hist) > prev_invasions:
            reward += REWARD_LANE_INVASION
        
        # 6. Collision penalty
        if len(self.collision_hist) > 0:
            reward += REWARD_COLLISION
        
        # 7. Bonus for good driving
        if waypoint and lane_offset < 0.5 and speed > 5.0:
            reward += 1.0
        
        # NEW: 8. Lane arrow following rewards
        if lane_changed:
            if lane_change_direction == self.target_lane_from_arrow:
                # Correct lane change following arrow!
                reward += REWARD_SUCCESSFUL_LANE_CHANGE
                print(f"  ✓ Successful {lane_change_direction} lane change following arrow!")
            else:
                # Wrong direction lane change
                reward += REWARD_WRONG_LANE_CHANGE
        
        # NEW: 9. Continuous reward for moving toward arrow direction
        if self.target_lane_from_arrow is not None and waypoint:
            # Small reward for steering in correct direction when arrow present
            if self.target_lane_from_arrow == "left" and self.last_steer < -0.05:
                reward += REWARD_FOLLOW_ARROW * 0.1  # Small continuous reward
            elif self.target_lane_from_arrow == "right" and self.last_steer > 0.05:
                reward += REWARD_FOLLOW_ARROW * 0.1
        
        return reward
    
    def _is_done(self):
        """Check termination"""
        if len(self.collision_hist) > 0:
            return True
        
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        if waypoint is None:
            return True
        
        if transform.location.distance(waypoint.transform.location) > 4.0:
            return True
        
        if self.steps >= MAX_STEPS:
            return True
        
        return False
    
    def _cleanup(self):
        """Cleanup"""
        if self.vehicle:
            self.vehicle.destroy()
        for sensor in self.sensors:
            if sensor.is_alive:
                sensor.destroy()
        self.sensors.clear()
        self.vehicle = None
    
    def close(self):
        """Close environment"""
        self._cleanup()
        settings = self.world.get_settings()
        settings.synchronous_mode = False
        self.world.apply_settings(settings)

# ======================= DQN AGENT =======================
class DQNAgent:
    def __init__(self, state_dim=18, action_dim=9):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.policy_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=LEARNING_RATE)
        self.memory = ReplayMemory(MEMORY_SIZE)
        
        self.epsilon = EPSILON_START
        self.action_dim = action_dim
    
    def select_action(self, state):
        """Epsilon-greedy action selection"""
        if random.random() < self.epsilon:
            return random.randrange(self.action_dim)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.policy_net(state_tensor)
            return q_values.argmax().item()
    
    def update(self):
        """Update network"""
        if len(self.memory) < BATCH_SIZE:
            return 0.0
        
        transitions = self.memory.sample(BATCH_SIZE)
        batch = list(zip(*transitions))
        
        state_batch = torch.FloatTensor(np.array(batch[0])).to(self.device)
        action_batch = torch.LongTensor(batch[1]).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch[2]).to(self.device)
        next_state_batch = torch.FloatTensor(np.array(batch[3])).to(self.device)
        done_batch = torch.FloatTensor(batch[4]).to(self.device)
        
        current_q = self.policy_net(state_batch).gather(1, action_batch)
        
        with torch.no_grad():
            next_actions = self.policy_net(next_state_batch).argmax(1, keepdim=True)
            next_q = self.target_net(next_state_batch).gather(1, next_actions).squeeze()
            target_q = reward_batch + (1 - done_batch) * GAMMA * next_q
        
        loss = nn.SmoothL1Loss()(current_q.squeeze(), target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def update_target_network(self):
        """Update target network"""
        self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def decay_epsilon(self):
        """Decay exploration"""
        self.epsilon = max(EPSILON_END, self.epsilon * EPSILON_DECAY)

# ======================= TRAINING =======================
def train():
    print("="*70)
    print("CARLA DQN TRAINING - SMOOTH DRIVING + LANE ARROW FOLLOWING")
    print("="*70)
    
    env = ImprovedCarlaEnv()
    agent = DQNAgent(state_dim=18, action_dim=9)
    
    episode_rewards = []
    episode_distances = []
    episode_lengths = []
    episode_lane_invasions = []
    episode_lane_changes = []
    
    print("\nStarting training...")
    print("Focus: Smooth control + Lane keeping + Following lane arrows\n")
    start_time = time.time()
    
    best_avg_distance = 0.0
    
    for episode in range(EPISODES):
        state = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)
            
            agent.memory.push(state, action, reward, next_state, float(done))
            loss = agent.update()
            
            episode_reward += reward
            state = next_state
        
        if (episode + 1) % TARGET_UPDATE == 0:
            agent.update_target_network()
        
        agent.decay_epsilon()
        
        # Track metrics
        episode_rewards.append(episode_reward)
        episode_distances.append(env.distance_traveled)
        episode_lengths.append(env.steps)
        episode_lane_invasions.append(len(env.lane_invasion_hist))
        episode_lane_changes.append(env.successful_lane_changes)
        
        # Logging
        if (episode + 1) % 10 == 0:
            avg_reward = np.mean(episode_rewards[-10:])
            avg_distance = np.mean(episode_distances[-10:])
            avg_length = np.mean(episode_lengths[-10:])
            avg_invasions = np.mean(episode_lane_invasions[-10:])
            avg_changes = np.mean(episode_lane_changes[-10:])
            elapsed = (time.time() - start_time) / 60
            
            print(f"Ep {episode+1:3d} | "
                  f"R: {episode_reward:7.1f} | "
                  f"D: {env.distance_traveled:6.1f}m | "
                  f"L: {env.steps:3d} | "
                  f"LI: {len(env.lane_invasion_hist):2d} | "
                  f"LC: {env.successful_lane_changes} | "
                  f"ε: {agent.epsilon:.3f} | "
                  f"Avg10: R={avg_reward:6.1f}, D={avg_distance:5.1f}m, LC={avg_changes:.1f} | "
                  f"T: {elapsed:.1f}min")
            
            if avg_distance > best_avg_distance:
                best_avg_distance = avg_distance
                torch.save(agent.policy_net.state_dict(), 'carla_dqn_arrow_best.pth')
                print(f"  ✓ New best model saved! Avg distance: {avg_distance:.1f}m")
        
        if (episode + 1) % 50 == 0:
            torch.save(agent.policy_net.state_dict(), f'dqn_arrow_ep{episode+1}.pth')
            print(f"  ✓ Checkpoint saved")
    
    torch.save(agent.policy_net.state_dict(), 'carla_dqn_arrow_final.pth')
    print(f"\n✓ Training completed in {(time.time() - start_time) / 60:.1f} minutes")
    print("✓ Final model saved: carla_dqn_arrow_final.pth")
    
    # Summary
    print("\n" + "="*70)
    print("TRAINING SUMMARY")
    print("="*70)
    print(f"Best avg distance (10 ep): {best_avg_distance:.1f}m")
    print(f"Final avg reward (last 20): {np.mean(episode_rewards[-20:]):.1f}")
    print(f"Final avg distance (last 20): {np.mean(episode_distances[-20:]):.1f}m")
    print(f"Final avg lane invasions (last 20): {np.mean(episode_lane_invasions[-20:]):.1f}")
    print(f"Final avg successful lane changes (last 20): {np.mean(episode_lane_changes[-20:]):.1f}")
    print("="*70)
    
    env.close()

if __name__ == "__main__":
    train()

## dqn_smooth_final

In [ ]:

"""
CARLA RL Training using DQN - IMPROVED VERSION
Focus: Smooth driving with better lane keeping

Key Improvements:
1. Better action space (gradual steering changes)
2. Enhanced state representation (heading error, curvature)
3. Reward shaping for smooth control
4. Action smoothing mechanism
"""

import carla
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import time

# ======================= CONFIG =======================
EPISODES = 300
MAX_STEPS = 500
MEMORY_SIZE = 50000
BATCH_SIZE = 128
GAMMA = 0.99
EPSILON_START = 1.0
EPSILON_END = 0.05
EPSILON_DECAY = 0.995
LEARNING_RATE = 1e-4
TARGET_UPDATE = 10

# IMPROVED DISCRETE ACTIONS (smoother, more gradual)
# Format: (throttle, steer)
ACTIONS = {
    0: (0.5, 0.0),    # Straight forward (moderate speed)
    1: (0.5, -0.15),  # Gentle left
    2: (0.5, 0.15),   # Gentle right
    3: (0.4, 0.0),    # Slow forward
    4: (0.4, -0.15),  # Slow + gentle left
    5: (0.4, 0.15),   # Slow + gentle right
    6: (0.0, 0.0),    # Brake/Stop
    7: (0.3, -0.3),   # Sharp left (for curves)
    8: (0.3, 0.3),    # Sharp right (for curves)
}

# IMPROVED REWARDS
REWARD_DISTANCE = 1.0          # Per meter forward
REWARD_SPEED = 1.5             # For maintaining target speed
REWARD_LANE_CENTER = 5.0       # Strong reward for staying centered
REWARD_HEADING = 3.0           # Reward for correct heading
REWARD_SMOOTHNESS = 2.0        # Reward for smooth control
REWARD_COLLISION = -200.0      # Heavy collision penalty
REWARD_OFF_ROAD = -100.0       # Off-road penalty
REWARD_LANE_INVASION = -20.0   # Lane crossing penalty
REWARD_JERKY = -5.0            # Penalty for sudden steering changes

TARGET_SPEED = 8.0  # m/s (more reasonable for town driving)

# ======================= DQN NETWORK =======================
class DQN(nn.Module):
    def __init__(self, state_dim=15, action_dim=9):
        super(DQN, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )
    
    def forward(self, x):
        return self.network(x)

# ======================= REPLAY MEMORY =======================
class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

# ======================= IMPROVED CARLA ENVIRONMENT =======================
class ImprovedCarlaEnv:
    def __init__(self, town='Town04', port=2000):
        self.client = carla.Client('localhost', port)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world(town)
        
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05  # Faster ticks for smoother control
        self.world.apply_settings(settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()
        self.vehicle = None
        self.sensors = []
        
        self.collision_hist = []
        self.lane_invasion_hist = []
        self.spawn_points = self.map.get_spawn_points()
        
        self.last_location = None
        self.distance_traveled = 0.0
        self.steps = 0
        
        # For smoothness tracking
        self.last_steer = 0.0
        self.last_throttle = 0.5
        self.steer_history = deque(maxlen=5)
        
        print("Improved Environment initialized!")
    
    def reset(self):
        """Reset environment"""
        self._cleanup()
        
        # Spawn vehicle
        vehicle_bp = self.blueprint_library.filter('vehicle.tesla.model3')[0]
        spawn_point = random.choice(self.spawn_points[:30])
        
        try:
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        except:
            spawn_point = random.choice(self.spawn_points)
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        
        # Collision sensor
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        collision_sensor.listen(lambda e: self.collision_hist.append(e))
        self.sensors.append(collision_sensor)
        
        # Lane invasion sensor
        lane_bp = self.blueprint_library.find('sensor.other.lane_invasion')
        lane_sensor = self.world.spawn_actor(
            lane_bp, carla.Transform(), attach_to=self.vehicle
        )
        lane_sensor.listen(lambda e: self.lane_invasion_hist.append(e))
        self.sensors.append(lane_sensor)
        
        # Reset tracking
        self.collision_hist.clear()
        self.lane_invasion_hist.clear()
        self.last_location = self.vehicle.get_location()
        self.distance_traveled = 0.0
        self.steps = 0
        self.last_steer = 0.0
        self.last_throttle = 0.5
        self.steer_history.clear()
        
        # Warmup
        for _ in range(10):
            self.world.tick()
        
        return self._get_state()
    
    def _get_state(self):
        """Enhanced state representation"""
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        waypoint = self.map.get_waypoint(
            transform.location, 
            project_to_road=True, 
            lane_type=carla.LaneType.Driving
        )
        
        if waypoint is None:
            return np.zeros(15, dtype=np.float32)
        
        # Lane offset (cross-track error)
        lane_center = waypoint.transform.location
        offset_vec = transform.location - lane_center
        right_vec = transform.get_right_vector()
        lane_offset = offset_vec.x * right_vec.x + offset_vec.y * right_vec.y
        
        # Heading error (critical for smooth driving)
        lane_yaw = waypoint.transform.rotation.yaw
        vehicle_yaw = transform.rotation.yaw
        heading_error = (vehicle_yaw - lane_yaw + 180) % 360 - 180
        heading_error_rad = np.radians(heading_error)
        
        # Look-ahead waypoint for anticipation
        next_waypoints = waypoint.next(5.0)
        if next_waypoints:
            next_wp = next_waypoints[0]
            wp_vec = next_wp.transform.location - transform.location
            forward_vec = transform.get_forward_vector()
            
            # Angle to next waypoint
            angle_to_wp = np.arctan2(
                wp_vec.y * forward_vec.x - wp_vec.x * forward_vec.y,
                wp_vec.x * forward_vec.x + wp_vec.y * forward_vec.y
            )
        else:
            angle_to_wp = 0.0
        
        # Road curvature (helps anticipate turns)
        curvature = 0.0
        if next_waypoints:
            next_wp = next_waypoints[0]
            curvature_yaw = next_wp.transform.rotation.yaw - lane_yaw
            curvature = np.sin(np.radians(curvature_yaw))
        
        # Traffic light state
        tl_state = 0.0  # 0: none, 0.33: green, 0.66: yellow, 1.0: red
        if self.vehicle.is_at_traffic_light():
            tl = self.vehicle.get_traffic_light()
            if tl:
                state = tl.get_state()
                if state == carla.TrafficLightState.Green:
                    tl_state = 0.33
                elif state == carla.TrafficLightState.Yellow:
                    tl_state = 0.66
                elif state == carla.TrafficLightState.Red:
                    tl_state = 1.0
        
        # Steering smoothness indicator
        steer_variance = np.std(list(self.steer_history)) if len(self.steer_history) > 2 else 0.0
        
        state = np.array([
            speed / 30.0,                          # Normalized speed
            lane_offset / 3.5,                     # Lane offset
            np.sin(heading_error_rad),             # Heading error (sin)
            np.cos(heading_error_rad),             # Heading error (cos)
            np.sin(angle_to_wp),                   # Angle to waypoint (sin)
            np.cos(angle_to_wp),                   # Angle to waypoint (cos)
            curvature,                             # Road curvature
            velocity.x / 30.0,                     # Velocity X
            velocity.y / 30.0,                     # Velocity Y
            float(waypoint.is_junction),           # Junction flag
            tl_state,                              # Traffic light state
            self.last_steer,                       # Previous steering
            steer_variance,                        # Steering smoothness
            self.distance_traveled / 1000.0,       # Total distance
            self.steps / 500.0                     # Normalized steps
        ], dtype=np.float32)
        
        return state
    
    def step(self, action_idx):
        """Execute action with smoothing"""
        throttle_raw, steer_raw = ACTIONS[action_idx]
        
        # Apply smoothing to reduce jerky movements
        alpha = 0.3  # Smoothing factor
        throttle = alpha * throttle_raw + (1 - alpha) * self.last_throttle
        steer = alpha * steer_raw + (1 - alpha) * self.last_steer
        
        # Limit steering rate of change
        max_steer_change = 0.1
        steer_change = np.clip(steer - self.last_steer, -max_steer_change, max_steer_change)
        steer = self.last_steer + steer_change
        
        # Store for next iteration
        self.last_steer = steer
        self.last_throttle = throttle
        self.steer_history.append(steer)
        
        # Apply control
        control = carla.VehicleControl(
            throttle=float(throttle),
            steer=float(steer),
            brake=0.0 if throttle > 0 else 0.5
        )
        
        self.vehicle.apply_control(control)
        self.world.tick()
        self.steps += 1
        
        # Track lane invasions
        prev_invasions = len(self.lane_invasion_hist)
        
        # Update distance
        current_location = self.vehicle.get_location()
        distance_step = 0.0
        if self.last_location:
            distance_step = current_location.distance(self.last_location)
            self.distance_traveled += distance_step
        self.last_location = current_location
        
        # Get new state
        next_state = self._get_state()
        
        # Calculate reward
        reward = self._calculate_reward(distance_step, prev_invasions, steer_change)
        
        # Check done
        done = self._is_done()
        
        return next_state, reward, done
    
    def _calculate_reward(self, distance_step, prev_invasions, steer_change):
        """Improved reward function for smooth driving"""
        reward = 0.0
        
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        # 1. Forward progress reward
        reward += REWARD_DISTANCE * distance_step
        
        # 2. Speed reward (Gaussian around target speed)
        speed_error = abs(speed - TARGET_SPEED)
        speed_reward = REWARD_SPEED * np.exp(-speed_error / 5.0)
        reward += speed_reward
        
        # 3. Lane keeping reward (critical!)
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        if waypoint:
            # Cross-track error
            lane_center = waypoint.transform.location
            offset_vec = transform.location - lane_center
            right_vec = transform.get_right_vector()
            lane_offset = abs(offset_vec.x * right_vec.x + offset_vec.y * right_vec.y)
            
            # Strong reward for staying centered (exponential decay)
            lane_reward = REWARD_LANE_CENTER * np.exp(-lane_offset * 3.0)
            reward += lane_reward
            
            # Heading alignment reward
            lane_yaw = waypoint.transform.rotation.yaw
            vehicle_yaw = transform.rotation.yaw
            heading_error = abs((vehicle_yaw - lane_yaw + 180) % 360 - 180)
            heading_reward = REWARD_HEADING * np.exp(-heading_error / 30.0)
            reward += heading_reward
        else:
            reward += REWARD_OFF_ROAD
        
        # 4. Smoothness reward (penalize jerky steering)
        steer_jerk = abs(steer_change)
        if steer_jerk > 0.05:  # Penalize large steering changes
            reward += REWARD_JERKY * steer_jerk
        else:
            reward += REWARD_SMOOTHNESS * (0.05 - steer_jerk)
        
        # 5. Lane invasion penalty
        if len(self.lane_invasion_hist) > prev_invasions:
            reward += REWARD_LANE_INVASION
        
        # 6. Collision penalty
        if len(self.collision_hist) > 0:
            reward += REWARD_COLLISION
        
        # 7. Bonus for maintaining lane over time
        if waypoint and lane_offset < 0.5 and speed > 5.0:
            reward += 1.0  # Bonus for good driving
        
        return reward
    
    def _is_done(self):
        """Check termination"""
        # Collision
        if len(self.collision_hist) > 0:
            return True
        
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        # Off road
        if waypoint is None:
            return True
        
        # Too far from lane center
        if transform.location.distance(waypoint.transform.location) > 4.0:
            return True
        
        # Max steps
        if self.steps >= MAX_STEPS:
            return True
        
        return False
    
    def _cleanup(self):
        """Cleanup"""
        if self.vehicle:
            self.vehicle.destroy()
        for sensor in self.sensors:
            if sensor.is_alive:
                sensor.destroy()
        self.sensors.clear()
        self.vehicle = None
    
    def close(self):
        """Close environment"""
        self._cleanup()
        settings = self.world.get_settings()
        settings.synchronous_mode = False
        self.world.apply_settings(settings)

# ======================= DQN AGENT =======================
class DQNAgent:
    def __init__(self, state_dim=15, action_dim=9):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.policy_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=LEARNING_RATE)
        self.memory = ReplayMemory(MEMORY_SIZE)
        
        self.epsilon = EPSILON_START
        self.action_dim = action_dim
    
    def select_action(self, state):
        """Epsilon-greedy action selection"""
        if random.random() < self.epsilon:
            return random.randrange(self.action_dim)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.policy_net(state_tensor)
            return q_values.argmax().item()
    
    def update(self):
        """Update network"""
        if len(self.memory) < BATCH_SIZE:
            return 0.0
        
        # Sample batch
        transitions = self.memory.sample(BATCH_SIZE)
        batch = list(zip(*transitions))
        
        state_batch = torch.FloatTensor(np.array(batch[0])).to(self.device)
        action_batch = torch.LongTensor(batch[1]).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch[2]).to(self.device)
        next_state_batch = torch.FloatTensor(np.array(batch[3])).to(self.device)
        done_batch = torch.FloatTensor(batch[4]).to(self.device)
        
        # Current Q values
        current_q = self.policy_net(state_batch).gather(1, action_batch)
        
        # Next Q values (Double DQN)
        with torch.no_grad():
            next_actions = self.policy_net(next_state_batch).argmax(1, keepdim=True)
            next_q = self.target_net(next_state_batch).gather(1, next_actions).squeeze()
            target_q = reward_batch + (1 - done_batch) * GAMMA * next_q
        
        # Huber loss (more stable than MSE)
        loss = nn.SmoothL1Loss()(current_q.squeeze(), target_q)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def update_target_network(self):
        """Update target network"""
        self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def decay_epsilon(self):
        """Decay exploration"""
        self.epsilon = max(EPSILON_END, self.epsilon * EPSILON_DECAY)

# ======================= TRAINING =======================
def train():
    print("="*70)
    print("IMPROVED CARLA DQN TRAINING - SMOOTH DRIVING")
    print("="*70)
    
    env = ImprovedCarlaEnv()
    agent = DQNAgent(state_dim=15, action_dim=9)
    
    episode_rewards = []
    episode_distances = []
    episode_lengths = []
    episode_lane_invasions = []
    
    print("\nStarting training...")
    print("Focus: Smooth control + Lane keeping\n")
    start_time = time.time()
    
    best_avg_distance = 0.0
    
    for episode in range(EPISODES):
        state = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            # Select and perform action
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)
            
            # Store transition
            agent.memory.push(state, action, reward, next_state, float(done))
            
            # Update network
            loss = agent.update()
            
            episode_reward += reward
            state = next_state
        
        # Update target network
        if (episode + 1) % TARGET_UPDATE == 0:
            agent.update_target_network()
        
        # Decay epsilon
        agent.decay_epsilon()
        
        # Track metrics
        episode_rewards.append(episode_reward)
        episode_distances.append(env.distance_traveled)
        episode_lengths.append(env.steps)
        episode_lane_invasions.append(len(env.lane_invasion_hist))
        
        # Logging
        if (episode + 1) % 10 == 0:
            avg_reward = np.mean(episode_rewards[-10:])
            avg_distance = np.mean(episode_distances[-10:])
            avg_length = np.mean(episode_lengths[-10:])
            avg_invasions = np.mean(episode_lane_invasions[-10:])
            elapsed = (time.time() - start_time) / 60
            
            print(f"Ep {episode+1:3d} | "
                  f"R: {episode_reward:7.1f} | "
                  f"D: {env.distance_traveled:6.1f}m | "
                  f"L: {env.steps:3d} | "
                  f"LI: {len(env.lane_invasion_hist):2d} | "
                  f"ε: {agent.epsilon:.3f} | "
                  f"Avg10: R={avg_reward:6.1f}, D={avg_distance:5.1f}m, LI={avg_invasions:.1f} | "
                  f"T: {elapsed:.1f}min")
            
            # Save best model
            if avg_distance > best_avg_distance:
                best_avg_distance = avg_distance
                torch.save(agent.policy_net.state_dict(), 'carla_dqn_smooth_best.pth')
                print(f"  ✓ New best model saved! Avg distance: {avg_distance:.1f}m")
        
        # Save checkpoint
        if (episode + 1) % 50 == 0:
            torch.save(agent.policy_net.state_dict(), f'dqn_smooth_ep{episode+1}.pth')
            print(f"  ✓ Checkpoint saved")
    
    # Save final model
    torch.save(agent.policy_net.state_dict(), 'carla_dqn_smooth_final.pth')
    print(f"\n✓ Training completed in {(time.time() - start_time) / 60:.1f} minutes")
    print("✓ Final model saved: carla_dqn_smooth_final.pth")
    
    # Summary
    print("\n" + "="*70)
    print("TRAINING SUMMARY")
    print("="*70)
    print(f"Best avg distance (10 ep): {best_avg_distance:.1f}m")
    print(f"Final avg reward (last 20): {np.mean(episode_rewards[-20:]):.1f}")
    print(f"Final avg distance (last 20): {np.mean(episode_distances[-20:]):.1f}m")
    print(f"Final avg lane invasions (last 20): {np.mean(episode_lane_invasions[-20:]):.1f}")
    print("="*70)
    
    env.close()

if __name__ == "__main__":
    train()

# carla_evlauation.py

## dqn_lane_arrow-evaluation

In [ ]:
"""
CARLA DQN Evaluation Script - Lane Arrow Following Version
Matches training with 18-dimensional state space
Features:
- Slower, smoother speed
- Fixed behind-vehicle camera
- Video generation
- Lane arrow detection and tracking
"""

import carla
import numpy as np
import torch
import torch.nn as nn
import time
import argparse
import os
import cv2
import imageio.v2 as imageio

# ACTIONS - Must match training
ACTIONS = {
    0: (0.5, 0.0),    # Straight forward
    1: (0.5, -0.15),  # Gentle left
    2: (0.5, 0.15),   # Gentle right
    3: (0.4, 0.0),    # Slow forward
    4: (0.4, -0.15),  # Slow + gentle left
    5: (0.4, 0.15),   # Slow + gentle right
    6: (0.0, 0.0),    # Brake/Stop
    7: (0.3, -0.3),   # Sharp left (for lane changes)
    8: (0.3, 0.3),    # Sharp right (for lane changes)
}

class DQN(nn.Module):
    def __init__(self, state_dim=18, action_dim=9):
        super(DQN, self).__init__()
        
        # MUST match training architecture (with dropout)
        self.network = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )
    
    def forward(self, x):
        return self.network(x)

class CarlaEvalEnv:
    def __init__(self, visualize=True):
        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world('Town04')
        
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()
        self.vehicle = None
        self.sensors = []
        self.camera = None
        self.visualize = visualize
        
        self.collision_hist = []
        self.lane_invasion_hist = []
        self.spawn_points = self.map.get_spawn_points()
        
        self.distance_traveled = 0.0
        self.last_location = None
        self.steps = 0
        self.speeds = []
        self.lane_offsets = []
        self.traffic_light_encounters = []
        
        # Video recording
        self.latest_image = None
        self.recording_frames = []
        
        # For state tracking
        self.last_steer = 0.0
        
        # Lane change tracking
        self.current_lane_id = None
        self.target_lane_from_arrow = None
        self.lane_change_in_progress = False
        self.successful_lane_changes = 0
        self.lane_arrows_detected = []
    
    def camera_callback(self, image):
        """Callback to capture camera frames for video"""
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]  # RGB to BGR
            self.latest_image = arr.copy()
            self.recording_frames.append(arr.copy())
        except Exception as e:
            print(f"Camera callback error: {e}")
    
    def _detect_lane_arrow(self, waypoint):
        """Detect if current lane has left/right arrow marking"""
        if waypoint is None:
            return None
        
        lane_change = waypoint.lane_change
        
        if lane_change == carla.LaneChange.Left:
            return "left"
        elif lane_change == carla.LaneChange.Right:
            return "right"
        elif lane_change == carla.LaneChange.Both:
            return "both"
        else:
            return None
    
    def _check_lane_availability(self, waypoint, direction):
        """Check if lane change to direction is safe and valid"""
        if waypoint is None or direction is None:
            return False
        
        try:
            if direction == "left":
                target_wp = waypoint.get_left_lane()
            elif direction == "right":
                target_wp = waypoint.get_right_lane()
            else:
                return False
            
            if target_wp is None:
                return False
            
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            if (waypoint.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            return True
            
        except Exception:
            return False
    
    def reset(self, spawn_idx=0):
        self._cleanup()
        
        vehicle_bp = self.blueprint_library.filter('vehicle.tesla.model3')[0]
        spawn_point = self.spawn_points[spawn_idx % len(self.spawn_points)]
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        
        # Collision sensor
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        collision_sensor.listen(lambda e: self.collision_hist.append(e))
        self.sensors.append(collision_sensor)
        
        # Lane invasion sensor
        lane_bp = self.blueprint_library.find('sensor.other.lane_invasion')
        lane_sensor = self.world.spawn_actor(
            lane_bp, carla.Transform(), attach_to=self.vehicle
        )
        lane_sensor.listen(lambda e: self.lane_invasion_hist.append(e))
        self.sensors.append(lane_sensor)
        
        # Camera for recording (mounted behind vehicle)
        cam_bp = self.blueprint_library.find('sensor.camera.rgb')
        cam_bp.set_attribute('image_size_x', '800')
        cam_bp.set_attribute('image_size_y', '600')
        cam_bp.set_attribute('fov', '90')
        cam_transform = carla.Transform(
            carla.Location(x=-5.0, z=2.5),
            carla.Rotation(pitch=-10)
        )
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)
        self.sensors.append(self.camera)
        
        self.collision_hist.clear()
        self.lane_invasion_hist.clear()
        self.distance_traveled = 0.0
        self.last_location = self.vehicle.get_location()
        self.steps = 0
        self.speeds = []
        self.lane_offsets = []
        self.traffic_light_encounters = []
        self.recording_frames = []
        self.last_steer = 0.0
        
        # Reset lane tracking
        self.current_lane_id = None
        self.target_lane_from_arrow = None
        self.lane_change_in_progress = False
        self.successful_lane_changes = 0
        self.lane_arrows_detected = []
        
        if self.visualize:
            spectator = self.world.get_spectator()
            t = self.vehicle.get_transform()
            spectator.set_transform(carla.Transform(
                t.location + carla.Location(x=-10, z=5),
                carla.Rotation(pitch=-20, yaw=t.rotation.yaw)
            ))
        
        for _ in range(10):
            self.world.tick()
        
        # Initialize lane ID
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint:
            self.current_lane_id = waypoint.lane_id
        
        return self._get_state()
    
    def _get_state(self):
        """Enhanced state representation - 18 dimensions"""
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        self.speeds.append(speed)
        
        waypoint = self.map.get_waypoint(
            transform.location, project_to_road=True, lane_type=carla.LaneType.Driving
        )
        
        if waypoint is None:
            return np.zeros(18, dtype=np.float32)
        
        # Lane offset (cross-track error)
        lane_center = waypoint.transform.location
        offset_vec = transform.location - lane_center
        right_vec = transform.get_right_vector()
        lane_offset = offset_vec.x * right_vec.x + offset_vec.y * right_vec.y
        self.lane_offsets.append(abs(lane_offset))
        
        # Heading error
        lane_yaw = waypoint.transform.rotation.yaw
        vehicle_yaw = transform.rotation.yaw
        heading_error = (vehicle_yaw - lane_yaw + 180) % 360 - 180
        heading_error_rad = np.radians(heading_error)
        
        # Look-ahead waypoint
        next_waypoints = waypoint.next(5.0)
        if next_waypoints:
            next_wp = next_waypoints[0]
            wp_vec = next_wp.transform.location - transform.location
            forward_vec = transform.get_forward_vector()
            angle_to_wp = np.arctan2(
                wp_vec.y * forward_vec.x - wp_vec.x * forward_vec.y,
                wp_vec.x * forward_vec.x + wp_vec.y * forward_vec.y
            )
        else:
            angle_to_wp = 0.0
        
        # Road curvature
        curvature = 0.0
        if next_waypoints:
            next_wp = next_waypoints[0]
            curvature_yaw = next_wp.transform.rotation.yaw - lane_yaw
            curvature = np.sin(np.radians(curvature_yaw))
        
        # Traffic light state
        tl_state = 0.0
        if self.vehicle.is_at_traffic_light():
            tl = self.vehicle.get_traffic_light()
            if tl:
                state_val = tl.get_state()
                self.traffic_light_encounters.append({
                    'state': str(state_val),
                    'speed': speed,
                    'step': self.steps
                })
                if state_val == carla.TrafficLightState.Green:
                    tl_state = 0.33
                elif state_val == carla.TrafficLightState.Yellow:
                    tl_state = 0.66
                elif state_val == carla.TrafficLightState.Red:
                    tl_state = 1.0
        
        # Steering smoothness
        steer_variance = 0.0
        
        # Lane arrow detection
        lane_arrow = self._detect_lane_arrow(waypoint)
        
        # Encode lane arrow
        arrow_left = 1.0 if lane_arrow in ["left", "both"] else 0.0
        arrow_right = 1.0 if lane_arrow in ["right", "both"] else 0.0
        arrow_straight = 1.0 if lane_arrow is None else 0.0
        
        # Check lane availability
        can_change_left = 1.0 if self._check_lane_availability(waypoint, "left") else 0.0
        can_change_right = 1.0 if self._check_lane_availability(waypoint, "right") else 0.0
        
        # Log arrow detection
        if lane_arrow is not None and lane_arrow not in [d['direction'] for d in self.lane_arrows_detected[-3:]]:
            self.lane_arrows_detected.append({
                'direction': lane_arrow,
                'step': self.steps,
                'location': (transform.location.x, transform.location.y)
            })
        
        # Update target from arrow
        if not self.lane_change_in_progress:
            if arrow_left and can_change_left:
                self.target_lane_from_arrow = "left"
            elif arrow_right and can_change_right:
                self.target_lane_from_arrow = "right"
            else:
                self.target_lane_from_arrow = None
        
        state = np.array([
            speed / 30.0,                          # 0: Normalized speed
            lane_offset / 3.5,                     # 1: Lane offset
            np.sin(heading_error_rad),             # 2: Heading error (sin)
            np.cos(heading_error_rad),             # 3: Heading error (cos)
            np.sin(angle_to_wp),                   # 4: Angle to waypoint (sin)
            np.cos(angle_to_wp),                   # 5: Angle to waypoint (cos)
            curvature,                             # 6: Road curvature
            velocity.x / 30.0,                     # 7: Velocity X
            velocity.y / 30.0,                     # 8: Velocity Y
            float(waypoint.is_junction),           # 9: Junction flag
            tl_state,                              # 10: Traffic light state
            self.last_steer,                       # 11: Previous steering
            steer_variance,                        # 12: Steering smoothness
            arrow_left,                            # 13: Lane arrow left
            arrow_right,                           # 14: Lane arrow right
            arrow_straight,                        # 15: No arrow (straight)
            can_change_left,                       # 16: Left lane available
            can_change_right,                      # 17: Right lane available
        ], dtype=np.float32)
        
        return state
    
    def step(self, action_idx):
        throttle, steer = ACTIONS[action_idx]
        
        # Store for state tracking
        self.last_steer = steer
        
        control = carla.VehicleControl(
            throttle=throttle,
            steer=steer,
            brake=0.0 if throttle > 0 else 0.3
        )
        
        self.vehicle.apply_control(control)
        
        if self.visualize:
            spectator = self.world.get_spectator()
            t = self.vehicle.get_transform()
            spec_location = t.location - t.get_forward_vector() * 10 + carla.Location(z=5)
            spectator.set_transform(carla.Transform(
                spec_location,
                carla.Rotation(pitch=-20, yaw=t.rotation.yaw)
            ))
        
        self.world.tick()
        self.steps += 1
        
        # Track lane changes
        prev_lane_id = self.current_lane_id
        
        current_location = self.vehicle.get_location()
        if self.last_location:
            self.distance_traveled += current_location.distance(self.last_location)
        self.last_location = current_location
        
        # Update current lane
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint:
            self.current_lane_id = waypoint.lane_id
        
        # Detect lane change completion
        if prev_lane_id is not None and self.current_lane_id != prev_lane_id:
            if self.current_lane_id > prev_lane_id:
                lane_change_direction = "left"
            else:
                lane_change_direction = "right"
            
            if lane_change_direction == self.target_lane_from_arrow:
                self.successful_lane_changes += 1
                print(f"    ✓ Lane change {lane_change_direction} at step {self.steps}")
            
            self.lane_change_in_progress = False
            self.target_lane_from_arrow = None
        
        # Detect lane change initiation
        if not self.lane_change_in_progress and abs(steer) > 0.2:
            self.lane_change_in_progress = True
        
        next_state = self._get_state()
        done = self._is_done()
        
        return next_state, done
    
    def _is_done(self):
        if len(self.collision_hist) > 0:
            return True
        
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        if waypoint is None:
            return True
        
        if transform.location.distance(waypoint.transform.location) > 4.0:
            return True
        
        return False
    
    def save_video(self, episode_num, output_dir="eval_videos"):
        """Save recorded frames as video"""
        if not self.recording_frames:
            print(f"  No frames to save for episode {episode_num}")
            return None
        
        os.makedirs(output_dir, exist_ok=True)
        video_path = os.path.join(output_dir, f"episode_{episode_num}.mp4")
        
        try:
            print(f"  Creating video from {len(self.recording_frames)} frames...")
            with imageio.get_writer(video_path, fps=20) as writer:
                for frame in self.recording_frames:
                    writer.append_data(frame)
            print(f"  ✓ Video saved: {video_path}")
            return video_path
        except Exception as e:
            print(f"  ✗ Video creation failed: {e}")
            return None
    
    def _cleanup(self):
        if self.vehicle:
            self.vehicle.destroy()
        for sensor in self.sensors:
            if sensor.is_alive:
                sensor.destroy()
        self.sensors.clear()
        self.vehicle = None
        self.camera = None
    
    def close(self):
        self._cleanup()
        settings = self.world.get_settings()
        settings.synchronous_mode = False
        self.world.apply_settings(settings)

def evaluate(model_path, num_episodes=5, max_steps=1000):
    print("="*70)
    print("CARLA DQN EVALUATION - LANE ARROW FOLLOWING")
    print("="*70)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DQN(state_dim=18, action_dim=9).to(device)  # 18 dims for arrow following
    
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
        print(f"✓ Model loaded: {model_path}\n")
    except Exception as e:
        print(f"✗ Failed to load model: {e}")
        return
    
    env = CarlaEvalEnv(visualize=True)
    
    all_distances = []
    all_speeds = []
    all_collisions = 0
    all_lane_invasions = 0
    all_lane_changes = 0
    all_arrows_detected = 0
    video_paths = []
    
    for ep in range(num_episodes):
        print(f"{'='*70}")
        print(f"EPISODE {ep+1}/{num_episodes}")
        print(f"{'='*70}")
        
        state = env.reset(spawn_idx=ep)
        done = False
        steps = 0
        
        print("Running... (Recording video + Tracking lane arrows)")
        print("Speed: SMOOTH for better control\n")
        action_counts = {i: 0 for i in range(9)}
        
        while not done and steps < max_steps:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            
            with torch.no_grad():
                q_values = model(state_tensor)
                action = q_values.argmax().item()
            
            action_counts[action] += 1
            next_state, done = env.step(action)
            state = next_state
            steps += 1
            
            if steps % 50 == 0:
                avg_speed = np.mean(env.speeds[-50:])
                avg_lane = np.mean(env.lane_offsets[-50:])
                print(f"  Step {steps:4d}: Speed={avg_speed:4.1f} m/s | "
                      f"Distance={env.distance_traveled:6.1f}m | "
                      f"Lane offset={avg_lane:4.2f}m | "
                      f"Lane changes={env.successful_lane_changes}")
        
        # Episode summary
        avg_speed = np.mean(env.speeds) if env.speeds else 0
        avg_lane_offset = np.mean(env.lane_offsets) if env.lane_offsets else 0
        max_lane_offset = max(env.lane_offsets) if env.lane_offsets else 0
        
        all_distances.append(env.distance_traveled)
        all_speeds.append(avg_speed)
        if len(env.collision_hist) > 0:
            all_collisions += 1
        all_lane_invasions += len(env.lane_invasion_hist)
        all_lane_changes += env.successful_lane_changes
        all_arrows_detected += len(env.lane_arrows_detected)
        
        print(f"\n{'─'*70}")
        print(f"EPISODE {ep+1} SUMMARY")
        print(f"{'─'*70}")
        print(f"Outcome: {'✗ COLLISION' if len(env.collision_hist) > 0 else '✓ SUCCESS'}")
        print(f"\nDriving Performance:")
        print(f"  • Distance Traveled: {env.distance_traveled:.1f} m")
        print(f"  • Average Speed: {avg_speed:.1f} m/s ({avg_speed*3.6:.1f} km/h)")
        print(f"  • Episode Duration: {steps} steps ({steps*0.05:.1f} seconds)")
        print(f"\nLane Keeping:")
        print(f"  • Average Lane Offset: {avg_lane_offset:.3f} m")
        print(f"  • Maximum Lane Offset: {max_lane_offset:.3f} m")
        print(f"  • Lane Invasions: {len(env.lane_invasion_hist)}")
        print(f"\nLane Arrow Following:")
        print(f"  • Arrows Detected: {len(env.lane_arrows_detected)}")
        print(f"  • Successful Lane Changes: {env.successful_lane_changes}")
        if env.lane_arrows_detected:
            print(f"  • Arrow Locations:")
            for arrow in env.lane_arrows_detected:
                print(f"    - {arrow['direction'].upper()} at step {arrow['step']}")
        print(f"\nSafety:")
        print(f"  • Collisions: {len(env.collision_hist)}")
        
        print(f"\nAction Distribution:")
        action_names = ['STRAIGHT', 'GENTLE_L', 'GENTLE_R', 'SLOW', 'SLOW_L', 'SLOW_R', 'BRAKE', 'SHARP_L', 'SHARP_R']
        for i, name in enumerate(action_names):
            pct = 100 * action_counts[i] / steps if steps > 0 else 0
            if pct > 1.0:  # Only show actions used >1%
                bar = '█' * int(pct / 2)
                print(f"  {name:10s}: {action_counts[i]:4d} ({pct:5.1f}%) {bar}")
        
        # Save video
        video_path = env.save_video(ep + 1)
        if video_path:
            video_paths.append(video_path)
        
        print()
        time.sleep(2)
    
    # Overall summary
    print(f"{'='*70}")
    print("OVERALL EVALUATION RESULTS")
    print(f"{'='*70}")
    
    success_rate = (num_episodes - all_collisions) / num_episodes
    avg_distance = np.mean(all_distances)
    avg_speed = np.mean(all_speeds)
    
    print(f"\nSuccess Rate: {success_rate*100:.1f}% ({num_episodes - all_collisions}/{num_episodes} episodes)")
    print(f"Average Distance: {avg_distance:.1f} m")
    print(f"Average Speed: {avg_speed:.1f} m/s ({avg_speed*3.6:.1f} km/h)")
    print(f"Total Collisions: {all_collisions}")
    print(f"Total Lane Invasions: {all_lane_invasions}")
    print(f"\nLane Arrow Following Performance:")
    print(f"Total Arrows Detected: {all_arrows_detected}")
    print(f"Total Successful Lane Changes: {all_lane_changes}")
    if all_arrows_detected > 0:
        print(f"Lane Change Success Rate: {(all_lane_changes / all_arrows_detected) * 100:.1f}%")
    
    # Video summary
    print(f"\n{'='*70}")
    print("VIDEO RECORDINGS")
    print(f"{'='*70}")
    print(f"Videos saved: {len(video_paths)}/{num_episodes}")
    for vp in video_paths:
        print(f"  • {vp}")
    
    # Performance assessment
    print(f"\n{'='*70}")
    print("PERFORMANCE ASSESSMENT")
    print(f"{'='*70}")
    
    speed_score = min(100, (avg_speed / 7.0) * 100)
    safety_score = success_rate * 100
    distance_score = min(100, (avg_distance / 500.0) * 100)
    
    # Lane arrow following score
    arrow_score = 0.0
    if all_arrows_detected > 0:
        arrow_score = min(100, (all_lane_changes / all_arrows_detected) * 100)
    
    overall_score = (speed_score + safety_score + distance_score + arrow_score) / 4
    
    print(f"\nSpeed Score:       {speed_score:5.1f}% (target: 7+ m/s)")
    print(f"Safety Score:      {safety_score:5.1f}% (no collisions)")
    print(f"Distance Score:    {distance_score:5.1f}% (target: 500+ m)")
    print(f"Arrow Follow Score: {arrow_score:5.1f}% (follow lane arrows)")
    print(f"Overall Score:     {overall_score:5.1f}%")
    
    if overall_score >= 85:
        grade = "A - Excellent autonomous driving with lane following!"
    elif overall_score >= 75:
        grade = "B - Good driving, minor improvements needed"
    elif overall_score >= 65:
        grade = "C - Fair, needs better arrow following"
    elif overall_score >= 50:
        grade = "D - Poor, inconsistent behavior"
    else:
        grade = "F - Failed to demonstrate autonomous driving"
    
    print(f"\nFINAL GRADE: {grade}")
    print(f"{'='*70}\n")
    
    env.close()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', default='carla_dqn_arrow_best.pth',
                       help='Path to trained model (default: carla_dqn_arrow_best.pth)')
    parser.add_argument('--episodes', type=int, default=5,
                       help='Number of episodes to evaluate (default: 5)')
    args = parser.parse_args()
    
    print("\nMake sure CARLA is running!\n")
    print("This evaluation tracks lane arrow following behavior.")
    print("The agent should change lanes when arrows indicate.\n")
    input("Press Enter to start...")
    
    evaluate(args.model, args.episodes)

## smooth_lane

In [ ]:
"""
CARLA DQN Evaluation Script - Modified Version
Changes:
- Slower, smoother speed
- Fixed behind-vehicle camera
- Video generation added
"""

import carla
import numpy as np
import torch
import torch.nn as nn
import time
import argparse
import os
import cv2
import imageio.v2 as imageio

# IMPROVED DISCRETE ACTIONS (matches training - smoother control)
# Format: (throttle, steer)
ACTIONS = {
    0: (0.5, 0.0),    # Straight forward (moderate speed)
    1: (0.5, -0.15),  # Gentle left
    2: (0.5, 0.15),   # Gentle right
    3: (0.4, 0.0),    # Slow forward
    4: (0.4, -0.15),  # Slow + gentle left
    5: (0.4, 0.15),   # Slow + gentle right
    6: (0.0, 0.0),    # Brake/Stop
    7: (0.3, -0.3),   # Sharp left (for curves)
    8: (0.3, 0.3),    # Sharp right (for curves)
}

class DQN(nn.Module):
    def __init__(self, state_dim=15, action_dim=9):
        super(DQN, self).__init__()
        
        # MUST match training architecture (with dropout)
        self.network = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )
    
    def forward(self, x):
        return self.network(x)

class CarlaEvalEnv:
    def __init__(self, visualize=True):
        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world('Town04')
        
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()
        self.vehicle = None
        self.sensors = []
        self.camera = None  # For video capture
        self.visualize = visualize
        
        self.collision_hist = []
        self.lane_invasion_hist = []
        self.spawn_points = self.map.get_spawn_points()
        
        self.distance_traveled = 0.0
        self.last_location = None
        self.steps = 0
        self.speeds = []
        self.lane_offsets = []
        self.traffic_light_encounters = []
        
        # Video recording
        self.latest_image = None
        self.recording_frames = []
        
        # For state tracking
        self.last_steer = 0.0
    
    def camera_callback(self, image):
        """Callback to capture camera frames for video"""
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]  # RGB to BGR
            self.latest_image = arr.copy()
            self.recording_frames.append(arr.copy())
        except Exception as e:
            print(f"Camera callback error: {e}")
    
    def reset(self, spawn_idx=0):
        self._cleanup()
        
        vehicle_bp = self.blueprint_library.filter('vehicle.tesla.model3')[0]
        spawn_point = self.spawn_points[spawn_idx % len(self.spawn_points)]
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        
        # Collision sensor
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        collision_sensor.listen(lambda e: self.collision_hist.append(e))
        self.sensors.append(collision_sensor)
        
        # Lane invasion sensor
        lane_bp = self.blueprint_library.find('sensor.other.lane_invasion')
        lane_sensor = self.world.spawn_actor(
            lane_bp, carla.Transform(), attach_to=self.vehicle
        )
        lane_sensor.listen(lambda e: self.lane_invasion_hist.append(e))
        self.sensors.append(lane_sensor)
        
        # Camera for recording (mounted behind vehicle)
        cam_bp = self.blueprint_library.find('sensor.camera.rgb')
        cam_bp.set_attribute('image_size_x', '800')
        cam_bp.set_attribute('image_size_y', '600')
        cam_bp.set_attribute('fov', '90')
        # Position: 5m behind, 2.5m up, slight downward pitch
        cam_transform = carla.Transform(
            carla.Location(x=-5.0, z=2.5),
            carla.Rotation(pitch=-10)
        )
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)
        self.sensors.append(self.camera)
        
        self.collision_hist.clear()
        self.lane_invasion_hist.clear()
        self.distance_traveled = 0.0
        self.last_location = self.vehicle.get_location()
        self.steps = 0
        self.speeds = []
        self.lane_offsets = []
        self.traffic_light_encounters = []
        self.recording_frames = []
        self.last_steer = 0.0
        
        if self.visualize:
            spectator = self.world.get_spectator()
            t = self.vehicle.get_transform()
            # FIXED: Spectator directly behind vehicle
            spectator.set_transform(carla.Transform(
                t.location + carla.Location(x=-10, z=5),
                carla.Rotation(pitch=-20, yaw=t.rotation.yaw)
            ))
        
        for _ in range(10):
            self.world.tick()
        
        return self._get_state()
    
    def _get_state(self):
        """Enhanced state representation - MUST match training (15 dims)"""
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        self.speeds.append(speed)
        
        waypoint = self.map.get_waypoint(
            transform.location, project_to_road=True, lane_type=carla.LaneType.Driving
        )
        
        if waypoint is None:
            return np.zeros(15, dtype=np.float32)
        
        # Lane offset (cross-track error)
        lane_center = waypoint.transform.location
        offset_vec = transform.location - lane_center
        right_vec = transform.get_right_vector()
        lane_offset = offset_vec.x * right_vec.x + offset_vec.y * right_vec.y
        self.lane_offsets.append(abs(lane_offset))
        
        # Heading error (critical for smooth driving)
        lane_yaw = waypoint.transform.rotation.yaw
        vehicle_yaw = transform.rotation.yaw
        heading_error = (vehicle_yaw - lane_yaw + 180) % 360 - 180
        heading_error_rad = np.radians(heading_error)
        
        # Look-ahead waypoint for anticipation
        next_waypoints = waypoint.next(5.0)
        if next_waypoints:
            next_wp = next_waypoints[0]
            wp_vec = next_wp.transform.location - transform.location
            forward_vec = transform.get_forward_vector()
            
            # Angle to next waypoint
            angle_to_wp = np.arctan2(
                wp_vec.y * forward_vec.x - wp_vec.x * forward_vec.y,
                wp_vec.x * forward_vec.x + wp_vec.y * forward_vec.y
            )
        else:
            angle_to_wp = 0.0
        
        # Road curvature (helps anticipate turns)
        curvature = 0.0
        if next_waypoints:
            next_wp = next_waypoints[0]
            curvature_yaw = next_wp.transform.rotation.yaw - lane_yaw
            curvature = np.sin(np.radians(curvature_yaw))
        
        # Traffic light state
        tl_state = 0.0  # 0: none, 0.33: green, 0.66: yellow, 1.0: red
        if self.vehicle.is_at_traffic_light():
            tl = self.vehicle.get_traffic_light()
            if tl:
                state_val = tl.get_state()
                self.traffic_light_encounters.append({
                    'state': str(state_val),
                    'speed': speed,
                    'step': self.steps
                })
                if state_val == carla.TrafficLightState.Green:
                    tl_state = 0.33
                elif state_val == carla.TrafficLightState.Yellow:
                    tl_state = 0.66
                elif state_val == carla.TrafficLightState.Red:
                    tl_state = 1.0
        
        # Steering smoothness (use last steer or default)
        last_steer = getattr(self, 'last_steer', 0.0)
        steer_variance = 0.0  # Can't compute in evaluation mode without history
        
        state = np.array([
            speed / 30.0,                          # 0: Normalized speed
            lane_offset / 3.5,                     # 1: Lane offset
            np.sin(heading_error_rad),             # 2: Heading error (sin)
            np.cos(heading_error_rad),             # 3: Heading error (cos)
            np.sin(angle_to_wp),                   # 4: Angle to waypoint (sin)
            np.cos(angle_to_wp),                   # 5: Angle to waypoint (cos)
            curvature,                             # 6: Road curvature
            velocity.x / 30.0,                     # 7: Velocity X
            velocity.y / 30.0,                     # 8: Velocity Y
            float(waypoint.is_junction),           # 9: Junction flag
            tl_state,                              # 10: Traffic light state
            last_steer,                            # 11: Previous steering
            steer_variance,                        # 12: Steering smoothness
            self.distance_traveled / 1000.0,       # 13: Total distance
            self.steps / 500.0                     # 14: Normalized steps
        ], dtype=np.float32)
        
        return state
    
    def step(self, action_idx):
        throttle, steer = ACTIONS[action_idx]
        
        # Store for state tracking
        self.last_steer = steer
        
        control = carla.VehicleControl(
            throttle=throttle,
            steer=steer,
            brake=0.0 if throttle > 0 else 0.3
        )
        
        self.vehicle.apply_control(control)
        
        if self.visualize:
            spectator = self.world.get_spectator()
            t = self.vehicle.get_transform()
            # FIXED: Keep spectator directly behind, no zigzag
            spec_location = t.location - t.get_forward_vector() * 10 + carla.Location(z=5)
            spectator.set_transform(carla.Transform(
                spec_location,
                carla.Rotation(pitch=-20, yaw=t.rotation.yaw)
            ))
        
        self.world.tick()
        self.steps += 1
        
        current_location = self.vehicle.get_location()
        if self.last_location:
            self.distance_traveled += current_location.distance(self.last_location)
        self.last_location = current_location
        
        next_state = self._get_state()
        done = self._is_done()
        
        return next_state, done
    
    def _is_done(self):
        if len(self.collision_hist) > 0:
            return True
        
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        if waypoint is None:
            return True
        
        if transform.location.distance(waypoint.transform.location) > 4.0:
            return True
        
        return False
    
    def save_video(self, episode_num, output_dir="eval_videos"):
        """Save recorded frames as video"""
        if not self.recording_frames:
            print(f"  No frames to save for episode {episode_num}")
            return None
        
        os.makedirs(output_dir, exist_ok=True)
        video_path = os.path.join(output_dir, f"episode_{episode_num}.mp4")
        
        try:
            print(f"  Creating video from {len(self.recording_frames)} frames...")
            with imageio.get_writer(video_path, fps=20) as writer:
                for frame in self.recording_frames:
                    writer.append_data(frame)
            print(f"  ✓ Video saved: {video_path}")
            return video_path
        except Exception as e:
            print(f"  ✗ Video creation failed: {e}")
            return None
    
    def _cleanup(self):
        if self.vehicle:
            self.vehicle.destroy()
        for sensor in self.sensors:
            if sensor.is_alive:
                sensor.destroy()
        self.sensors.clear()
        self.vehicle = None
        self.camera = None
    
    def close(self):
        self._cleanup()
        settings = self.world.get_settings()
        settings.synchronous_mode = False
        self.world.apply_settings(settings)

def evaluate(model_path, num_episodes=5, max_steps=1000):
    print("="*70)
    print("CARLA DQN AGENT EVALUATION (SMOOTH & SLOW)")
    print("="*70)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DQN(state_dim=15, action_dim=9).to(device)  # Changed to 15 dims
    
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
        print(f"✓ Model loaded: {model_path}\n")
    except Exception as e:
        print(f"✗ Failed to load model: {e}")
        return
    
    env = CarlaEvalEnv(visualize=True)
    
    all_distances = []
    all_speeds = []
    all_collisions = 0
    all_lane_invasions = 0
    video_paths = []
    
    for ep in range(num_episodes):
        print(f"{'='*70}")
        print(f"EPISODE {ep+1}/{num_episodes}")
        print(f"{'='*70}")
        
        state = env.reset(spawn_idx=ep)
        done = False
        steps = 0
        
        print("Running... (Recording video)")
        print("Speed: REDUCED for smooth evaluation\n")
        action_counts = {i: 0 for i in range(9)}
        
        while not done and steps < max_steps:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            
            with torch.no_grad():
                q_values = model(state_tensor)
                action = q_values.argmax().item()
            
            action_counts[action] += 1
            next_state, done = env.step(action)
            state = next_state
            steps += 1
            
            if steps % 50 == 0:
                avg_speed = np.mean(env.speeds[-50:])
                avg_lane = np.mean(env.lane_offsets[-50:])
                print(f"  Step {steps:4d}: Speed={avg_speed:4.1f} m/s | "
                      f"Distance={env.distance_traveled:6.1f}m | "
                      f"Lane offset={avg_lane:4.2f}m")
        
        # Episode summary
        avg_speed = np.mean(env.speeds) if env.speeds else 0
        avg_lane_offset = np.mean(env.lane_offsets) if env.lane_offsets else 0
        max_lane_offset = max(env.lane_offsets) if env.lane_offsets else 0
        
        all_distances.append(env.distance_traveled)
        all_speeds.append(avg_speed)
        if len(env.collision_hist) > 0:
            all_collisions += 1
        all_lane_invasions += len(env.lane_invasion_hist)
        
        print(f"\n{'─'*70}")
        print(f"EPISODE {ep+1} SUMMARY")
        print(f"{'─'*70}")
        print(f"Outcome: {'✗ COLLISION' if len(env.collision_hist) > 0 else '✓ SUCCESS'}")
        print(f"\nDriving Performance:")
        print(f"  • Distance Traveled: {env.distance_traveled:.1f} m")
        print(f"  • Average Speed: {avg_speed:.1f} m/s ({avg_speed*3.6:.1f} km/h)")
        print(f"  • Episode Duration: {steps} steps ({steps*0.05:.1f} seconds)")
        print(f"\nLane Keeping:")
        print(f"  • Average Lane Offset: {avg_lane_offset:.3f} m")
        print(f"  • Maximum Lane Offset: {max_lane_offset:.3f} m")
        print(f"  • Lane Invasions: {len(env.lane_invasion_hist)}")
        print(f"\nSafety:")
        print(f"  • Collisions: {len(env.collision_hist)}")
        
        # Save video for this episode
        video_path = env.save_video(ep + 1)
        if video_path:
            video_paths.append(video_path)
        
        print()
        time.sleep(2)
    
    # Overall summary
    print(f"{'='*70}")
    print("OVERALL EVALUATION RESULTS")
    print(f"{'='*70}")
    
    success_rate = (num_episodes - all_collisions) / num_episodes
    avg_distance = np.mean(all_distances)
    avg_speed = np.mean(all_speeds)
    
    print(f"\nSuccess Rate: {success_rate*100:.1f}% ({num_episodes - all_collisions}/{num_episodes} episodes)")
    print(f"Average Distance: {avg_distance:.1f} m")
    print(f"Average Speed: {avg_speed:.1f} m/s ({avg_speed*3.6:.1f} km/h)")
    print(f"Total Collisions: {all_collisions}")
    print(f"Total Lane Invasions: {all_lane_invasions}")
    
    # Video summary
    print(f"\n{'='*70}")
    print("VIDEO RECORDINGS")
    print(f"{'='*70}")
    print(f"Videos saved: {len(video_paths)}/{num_episodes}")
    for vp in video_paths:
        print(f"  • {vp}")
    
    # Performance assessment
    print(f"\n{'='*70}")
    print("PERFORMANCE ASSESSMENT")
    print(f"{'='*70}")
    
    # Adjusted scoring for slower speeds
    speed_score = min(100, (avg_speed / 7.0) * 100)  # Target 7 m/s instead of 10
    safety_score = success_rate * 100
    distance_score = min(100, (avg_distance / 500.0) * 100)
    overall_score = (speed_score + safety_score + distance_score) / 3
    
    print(f"\nSpeed Score:    {speed_score:5.1f}% (target: 7+ m/s for smooth driving)")
    print(f"Safety Score:   {safety_score:5.1f}% (no collisions)")
    print(f"Distance Score: {distance_score:5.1f}% (target: 500+ m)")
    print(f"Overall Score:  {overall_score:5.1f}%")
    
    if overall_score >= 80:
        grade = "A - Excellent smooth autonomous driving!"
    elif overall_score >= 70:
        grade = "B - Good, minor improvements needed"
    elif overall_score >= 60:
        grade = "C - Fair, needs work on consistency"
    elif overall_score >= 50:
        grade = "D - Poor, basic driving but unsafe"
    else:
        grade = "F - Failed to demonstrate autonomous driving"
    
    print(f"\nFINAL GRADE: {grade}")
    print(f"{'='*70}\n")
    
    env.close()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', default='carla_dqn_agent_final.pth')
    parser.add_argument('--episodes', type=int, default=5)
    args = parser.parse_args()
    
    print("\nMake sure CARLA is running!\n")
    input("Press Enter to start...")
    
    evaluate(args.model, args.episodes)

# PPO

## training


In [ ]:
"""
CARLA PPO Training - Optimized for Lane Keeping + Turns
Complete solution with proper reward shaping and real-time plotting

Task Requirements:
1. Maintain lane center while driving
2. Detect and follow lane change arrows
3. Execute smooth turns
4. Return to lane center after turns

Model: PPO (better than DQN for continuous control)
"""

import carla
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
import matplotlib.pyplot as plt
from collections import deque
import time
import os

# ==================== CONFIGURATION ====================
EPISODES = 500
MAX_STEPS = 800
BATCH_SIZE = 128
GAMMA = 0.99
GAE_LAMBDA = 0.95
CLIP_EPSILON = 0.2
LEARNING_RATE = 3e-4
ENTROPY_COEF = 0.01
VALUE_COEF = 0.5
UPDATE_EPOCHS = 10
TARGET_SPEED = 7.0  # m/s

# ==================== PPO ACTOR-CRITIC ====================
class ActorCritic(nn.Module):
    def __init__(self, state_dim=12, action_dim=2):
        super(ActorCritic, self).__init__()
        
        # Shared feature extractor
        self.shared = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )
        
        # Actor head (policy) - outputs mean and log_std for continuous actions
        self.actor_mean = nn.Linear(256, action_dim)
        self.actor_log_std = nn.Parameter(torch.zeros(action_dim))
        
        # Critic head (value function)
        self.critic = nn.Linear(256, 1)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
                nn.init.constant_(m.bias, 0.0)
        
        # Special init for policy head (smaller initial actions)
        nn.init.orthogonal_(self.actor_mean.weight, gain=0.01)
    
    def forward(self, state):
        features = self.shared(state)
        return features
    
    def act(self, state):
        features = self.forward(state)
        
        # Policy
        action_mean = self.actor_mean(features)
        action_std = torch.exp(self.actor_log_std)
        dist = Normal(action_mean, action_std)
        action = dist.sample()
        action_log_prob = dist.log_prob(action).sum(dim=-1)
        
        # Value
        value = self.critic(features)
        
        return action, action_log_prob, value
    
    def evaluate(self, state, action):
        features = self.forward(state)
        
        # Policy
        action_mean = self.actor_mean(features)
        action_std = torch.exp(self.actor_log_std)
        dist = Normal(action_mean, action_std)
        action_log_prob = dist.log_prob(action).sum(dim=-1)
        entropy = dist.entropy().sum(dim=-1)
        
        # Value
        value = self.critic(features)
        
        return action_log_prob, entropy, value


# ==================== PPO MEMORY ====================
class PPOMemory:
    def __init__(self):
        self.states = []
        self.actions = []
        self.log_probs = []
        self.rewards = []
        self.values = []
        self.dones = []
    
    def store(self, state, action, log_prob, reward, value, done):
        self.states.append(state)
        self.actions.append(action)
        self.log_probs.append(log_prob)
        self.rewards.append(reward)
        self.values.append(value)
        self.dones.append(done)
    
    def clear(self):
        self.states = []
        self.actions = []
        self.log_probs = []
        self.rewards = []
        self.values = []
        self.dones = []
    
    def get(self):
        return (
            torch.FloatTensor(np.array(self.states)),
            torch.FloatTensor(np.array(self.actions)),
            torch.FloatTensor(np.array(self.log_probs)),
            torch.FloatTensor(np.array(self.rewards)),
            torch.FloatTensor(np.array(self.values)),
            torch.FloatTensor(np.array(self.dones))
        )


# ==================== CARLA ENVIRONMENT ====================
class CarlaLaneKeepingEnv:
    def __init__(self):
        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world('Town04')
        
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()
        self.vehicle = None
        self.sensors = []
        
        self.collision_hist = []
        self.spawn_points = self.map.get_spawn_points()
        
        self.distance_traveled = 0.0
        self.last_location = None
        self.steps = 0
        
        # For reward calculation
        self.last_lane_offset = 0.0
        self.last_heading_error = 0.0
    
    def reset(self):
        self._cleanup()
        
        vehicle_bp = self.blueprint_library.filter('vehicle.tesla.model3')[0]
        spawn_point = np.random.choice(self.spawn_points[:30])
        
        try:
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        except:
            spawn_point = np.random.choice(self.spawn_points)
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        
        # Collision sensor
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        collision_sensor.listen(lambda e: self.collision_hist.append(e))
        self.sensors.append(collision_sensor)
        
        # Reset tracking
        self.collision_hist.clear()
        self.last_location = self.vehicle.get_location()
        self.distance_traveled = 0.0
        self.steps = 0
        
        self.last_lane_offset = 0.0
        self.last_heading_error = 0.0
        
        # Warmup
        for _ in range(10):
            self.world.tick()
        
        return self._get_state()
    
    def _get_state(self):
        """12-dimensional state vector optimized for lane keeping"""
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        waypoint = self.map.get_waypoint(
            transform.location, project_to_road=True, lane_type=carla.LaneType.Driving
        )
        
        if not waypoint:
            return np.zeros(12, dtype=np.float32)
        
        # 1. Lane offset (most important!)
        lane_center = waypoint.transform.location
        offset_vec = transform.location - lane_center
        right_vec = transform.get_right_vector()
        lane_offset = offset_vec.x * right_vec.x + offset_vec.y * right_vec.y
        
        # 2. Heading error
        lane_yaw = waypoint.transform.rotation.yaw
        vehicle_yaw = transform.rotation.yaw
        heading_error = (vehicle_yaw - lane_yaw + 180) % 360 - 180
        heading_error_rad = np.radians(heading_error)
        
        # 3. Look-ahead waypoint
        next_wps = waypoint.next(10.0)
        if next_wps:
            next_wp = next_wps[0]
            wp_vec = next_wp.transform.location - transform.location
            forward_vec = transform.get_forward_vector()
            angle_to_wp = np.arctan2(
                wp_vec.y * forward_vec.x - wp_vec.x * forward_vec.y,
                wp_vec.x * forward_vec.x + wp_vec.y * forward_vec.y
            )
        else:
            angle_to_wp = 0.0
        
        # 4. Curvature
        curvature = 0.0
        if next_wps:
            curvature_yaw = next_wps[0].transform.rotation.yaw - lane_yaw
            curvature = np.sin(np.radians(curvature_yaw))
        
        # 5. Speed normalized
        speed_norm = speed / 15.0
        
        # 6. Velocity components
        vel_x = velocity.x / 15.0
        vel_y = velocity.y / 15.0
        
        # 7. Junction detection
        is_junction = 1.0 if waypoint.is_junction else 0.0
        
        state = np.array([
            lane_offset / 3.0,           # 0: Lane offset (normalized)
            np.sin(heading_error_rad),   # 1: Heading error sin
            np.cos(heading_error_rad),   # 2: Heading error cos
            np.sin(angle_to_wp),         # 3: Angle to waypoint sin
            np.cos(angle_to_wp),         # 4: Angle to waypoint cos
            curvature,                   # 5: Road curvature
            speed_norm,                  # 6: Speed
            vel_x,                       # 7: Velocity X
            vel_y,                       # 8: Velocity Y
            is_junction,                 # 9: Junction flag
            lane_offset / 3.0 - self.last_lane_offset / 3.0,  # 10: Lane offset change
            heading_error_rad - self.last_heading_error,      # 11: Heading change
        ], dtype=np.float32)
        
        self.last_lane_offset = lane_offset
        self.last_heading_error = heading_error_rad
        
        return state
    
    def step(self, action):
        """
        Action: [steering, throttle_delta]
        steering: [-1, 1]
        throttle_delta: [-1, 1] (added to base throttle)
        """
        steering = float(np.clip(action[0], -0.4, 0.4))  # Limit max steering
        
        # Base throttle for target speed, action modulates it
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        # Simple speed controller
        speed_error = TARGET_SPEED - speed
        base_throttle = 0.5 + 0.1 * speed_error
        throttle_delta = float(np.clip(action[1], -0.3, 0.3))
        throttle = float(np.clip(base_throttle + throttle_delta, 0.0, 0.8))
        
        # Apply control
        control = carla.VehicleControl(
            throttle=throttle,
            steer=steering,
            brake=0.0
        )
        
        self.vehicle.apply_control(control)
        self.world.tick()
        self.steps += 1
        
        # Update distance
        current_location = self.vehicle.get_location()
        if self.last_location:
            self.distance_traveled += current_location.distance(self.last_location)
        self.last_location = current_location
        
        # Get next state
        next_state = self._get_state()
        
        # Calculate reward
        reward = self._calculate_reward(next_state, steering, throttle)
        
        # Check done
        done = self._is_done()
        
        return next_state, reward, done
    
    def _calculate_reward(self, state, steering, throttle):
        """Reward function optimized for lane keeping"""
        reward = 0.0
        
        # Extract state components
        lane_offset = state[0] * 3.0  # Denormalize
        heading_sin = state[1]
        heading_cos = state[2]
        heading_error = np.arctan2(heading_sin, heading_cos)
        curvature = state[5]
        speed = state[6] * 15.0
        
        # 1. LANE CENTERING (most important!) - Exponential reward
        lane_reward = 5.0 * np.exp(-3.0 * abs(lane_offset))
        reward += lane_reward
        
        # 2. HEADING ALIGNMENT - Stay parallel to lane
        heading_reward = 3.0 * np.exp(-2.0 * abs(heading_error))
        reward += heading_reward
        
        # 3. FORWARD PROGRESS - Reward distance traveled
        distance_reward = 0.1  # Small constant reward for each step
        reward += distance_reward
        
        # 4. SPEED MAINTENANCE - Stay near target speed
        speed_error = abs(speed - TARGET_SPEED)
        speed_reward = 2.0 * np.exp(-speed_error / 3.0)
        reward += speed_reward
        
        # 5. SMOOTH STEERING - Penalize jerky movements
        steering_penalty = -0.5 * abs(steering)
        reward += steering_penalty
        
        # 6. PENALTIES
        # Severe off-center penalty
        if abs(lane_offset) > 1.0:
            reward -= 5.0
        
        # Collision penalty
        if len(self.collision_hist) > 0:
            reward -= 100.0
        
        # Off-road penalty
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint is None or transform.location.distance(waypoint.transform.location) > 3.5:
            reward -= 50.0
        
        return reward
    
    def _is_done(self):
        # Collision
        if len(self.collision_hist) > 0:
            return True
        
        # Off road
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint is None:
            return True
        if transform.location.distance(waypoint.transform.location) > 3.5:
            return True
        
        # Max steps
        if self.steps >= MAX_STEPS:
            return True
        
        return False
    
    def _cleanup(self):
        if self.vehicle:
            self.vehicle.destroy()
        for sensor in self.sensors:
            if sensor.is_alive:
                sensor.destroy()
        self.sensors.clear()
        self.vehicle = None
    
    def close(self):
        self._cleanup()
        settings = self.world.get_settings()
        settings.synchronous_mode = False
        self.world.apply_settings(settings)


# ==================== PPO AGENT ====================
class PPOAgent:
    def __init__(self, state_dim=12, action_dim=2):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.policy = ActorCritic(state_dim, action_dim).to(self.device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=LEARNING_RATE)
        
        self.memory = PPOMemory()
    
    def select_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            action, log_prob, value = self.policy.act(state_tensor)
        
        return action.cpu().numpy()[0], log_prob.cpu().item(), value.cpu().item()
    
    def update(self):
        # Get data from memory
        states, actions, old_log_probs, rewards, values, dones = self.memory.get()
        
        states = states.to(self.device)
        actions = actions.to(self.device)
        old_log_probs = old_log_probs.to(self.device)
        
        # Compute advantages using GAE
        advantages = []
        returns = []
        
        gae = 0
        next_value = 0
        
        for t in reversed(range(len(rewards))):
            if t == len(rewards) - 1:
                next_value = 0
                next_done = 0
            else:
                next_value = values[t + 1]
                next_done = dones[t + 1]
            
            delta = rewards[t] + GAMMA * next_value * (1 - next_done) - values[t]
            gae = delta + GAMMA * GAE_LAMBDA * (1 - next_done) * gae
            
            advantages.insert(0, gae)
            returns.insert(0, gae + values[t])
        
        advantages = torch.FloatTensor(advantages).to(self.device)
        returns = torch.FloatTensor(returns).to(self.device)
        
        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # PPO update
        total_policy_loss = 0
        total_value_loss = 0
        total_entropy = 0
        
        for _ in range(UPDATE_EPOCHS):
            # Evaluate
            log_probs, entropy, state_values = self.policy.evaluate(states, actions)
            
            # Ratio
            ratios = torch.exp(log_probs - old_log_probs)
            
            # Surrogate losses
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - CLIP_EPSILON, 1 + CLIP_EPSILON) * advantages
            
            # Final loss
            policy_loss = -torch.min(surr1, surr2).mean()
            value_loss = 0.5 * ((returns - state_values.squeeze()) ** 2).mean()
            entropy_loss = -entropy.mean()
            
            loss = policy_loss + VALUE_COEF * value_loss + ENTROPY_COEF * entropy_loss
            
            # Backprop
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.policy.parameters(), 0.5)
            self.optimizer.step()
            
            total_policy_loss += policy_loss.item()
            total_value_loss += value_loss.item()
            total_entropy += entropy_loss.item()
        
        # Clear memory
        self.memory.clear()
        
        return {
            'policy_loss': total_policy_loss / UPDATE_EPOCHS,
            'value_loss': total_value_loss / UPDATE_EPOCHS,
            'entropy': -total_entropy / UPDATE_EPOCHS
        }


# ==================== TRAINING ====================
def train():
    print("="*80)
    print("PPO TRAINING - LANE KEEPING + TURNS")
    print("="*80)
    print(f"Episodes: {EPISODES}")
    print(f"Max steps per episode: {MAX_STEPS}")
    print(f"Target speed: {TARGET_SPEED} m/s\n")
    
    env = CarlaLaneKeepingEnv()
    agent = PPOAgent(state_dim=12, action_dim=2)
    
    # Tracking
    episode_rewards = []
    episode_distances = []
    episode_lengths = []
    policy_losses = []
    value_losses = []
    
    # Setup plotting
    plt.ion()
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    
    best_distance = 0.0
    
    print("Starting training...\n")
    start_time = time.time()
    
    for episode in range(EPISODES):
        state = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            # Select action
            action, log_prob, value = agent.select_action(state)
            
            # Take step
            next_state, reward, done = env.step(action)
            
            # Store in memory
            agent.memory.store(state, action, log_prob, reward, value, done)
            
            episode_reward += reward
            state = next_state
        
        # Update policy
        losses = agent.update()
        
        # Track metrics
        episode_rewards.append(episode_reward)
        episode_distances.append(env.distance_traveled)
        episode_lengths.append(env.steps)
        policy_losses.append(losses['policy_loss'])
        value_losses.append(losses['value_loss'])
        
        # Logging
        if (episode + 1) % 10 == 0:
            avg_reward = np.mean(episode_rewards[-10:])
            avg_distance = np.mean(episode_distances[-10:])
            avg_length = np.mean(episode_lengths[-10:])
            elapsed = (time.time() - start_time) / 60
            
            print(f"Ep {episode+1:3d} | "
                  f"R: {episode_reward:7.1f} | "
                  f"D: {env.distance_traveled:6.1f}m | "
                  f"L: {env.steps:3d} | "
                  f"Avg10: R={avg_reward:6.1f}, D={avg_distance:5.1f}m | "
                  f"Loss: P={losses['policy_loss']:.3f}, V={losses['value_loss']:.3f} | "
                  f"T: {elapsed:.1f}min")
            
            # Save best model
            if avg_distance > best_distance:
                best_distance = avg_distance
                torch.save(agent.policy.state_dict(), 'ppo_lane_keeping_best.pth')
                print(f"  ✓ New best! Avg distance: {avg_distance:.1f}m")
        
        # Update plots
        if (episode + 1) % 5 == 0:
            ax1.clear()
            ax1.plot(episode_rewards, alpha=0.3)
            ax1.plot(np.convolve(episode_rewards, np.ones(10)/10, mode='valid'))
            ax1.set_title('Episode Rewards')
            ax1.set_xlabel('Episode')
            ax1.set_ylabel('Reward')
            ax1.grid(True)
            
            ax2.clear()
            ax2.plot(episode_distances, alpha=0.3)
            ax2.plot(np.convolve(episode_distances, np.ones(10)/10, mode='valid'))
            ax2.set_title('Distance Traveled')
            ax2.set_xlabel('Episode')
            ax2.set_ylabel('Distance (m)')
            ax2.grid(True)
            
            ax3.clear()
            ax3.plot(policy_losses, label='Policy Loss')
            ax3.plot(value_losses, label='Value Loss')
            ax3.set_title('Training Losses')
            ax3.set_xlabel('Episode')
            ax3.set_ylabel('Loss')
            ax3.legend()
            ax3.grid(True)
            
            ax4.clear()
            ax4.plot(episode_lengths, alpha=0.3)
            ax4.plot(np.convolve(episode_lengths, np.ones(10)/10, mode='valid'))
            ax4.set_title('Episode Length')
            ax4.set_xlabel('Episode')
            ax4.set_ylabel('Steps')
            ax4.grid(True)
            
            plt.tight_layout()
            plt.pause(0.01)
            
            # Save plot
            plt.savefig('ppo_training_progress.png', dpi=150)
        
        # Save checkpoint
        if (episode + 1) % 50 == 0:
            torch.save(agent.policy.state_dict(), f'ppo_checkpoint_ep{episode+1}.pth')
    
    # Save final model
    torch.save(agent.policy.state_dict(), 'ppo_lane_keeping_final.pth')
    
    print(f"\n✓ Training completed in {(time.time() - start_time) / 60:.1f} minutes")
    print(f"✓ Best avg distance: {best_distance:.1f}m")
    print(f"✓ Models saved")
    
    plt.ioff()
    plt.show()
    
    env.close()


if __name__ == "__main__":
    train()

for training: /home/robotuser/carla_0.9.12/PythonAPI/examples/PPO_best_git/ppo_lane_keeping_final.pth


In [ ]:
"""
Continue PPO Training from Checkpoint
Loads your existing trained model and trains for 500 more episodes

Expected improvements after 500 more episodes:
- Distance: 923m → 1100-1300m
- Lane offset: 0.175m → 0.12-0.15m  
- Speed: 5.0 m/s → 6.0-6.5 m/s
"""

import carla
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
import matplotlib.pyplot as plt
import time
import os

# ==================== CONFIGURATION ====================
EPISODES = 500  # Train 500 MORE episodes
MAX_STEPS = 800
GAMMA = 0.99
GAE_LAMBDA = 0.95
CLIP_EPSILON = 0.2
LEARNING_RATE = 3e-4
ENTROPY_COEF = 0.01
VALUE_COEF = 0.5
UPDATE_EPOCHS = 10
TARGET_SPEED = 7.0

# CHECKPOINT TO LOAD
CHECKPOINT_PATH = 'ppo_lane_keeping_best.pth'
START_EPISODE = 500  # Your training ended at episode 500

# ==================== COPY YOUR EXISTING CLASSES ====================
# (ActorCritic, PPOMemory, CarlaLaneKeepingEnv, PPOAgent)
# Exactly the same as your training script

class ActorCritic(nn.Module):
    def __init__(self, state_dim=12, action_dim=2):
        super(ActorCritic, self).__init__()
        
        self.shared = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )
        
        self.actor_mean = nn.Linear(256, action_dim)
        self.actor_log_std = nn.Parameter(torch.zeros(action_dim))
        self.critic = nn.Linear(256, 1)
        
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
                nn.init.constant_(m.bias, 0.0)
        nn.init.orthogonal_(self.actor_mean.weight, gain=0.01)
    
    def forward(self, state):
        return self.shared(state)
    
    def act(self, state):
        features = self.forward(state)
        action_mean = self.actor_mean(features)
        action_std = torch.exp(self.actor_log_std)
        dist = Normal(action_mean, action_std)
        action = dist.sample()
        action_log_prob = dist.log_prob(action).sum(dim=-1)
        value = self.critic(features)
        return action, action_log_prob, value
    
    def evaluate(self, state, action):
        features = self.forward(state)
        action_mean = self.actor_mean(features)
        action_std = torch.exp(self.actor_log_std)
        dist = Normal(action_mean, action_std)
        action_log_prob = dist.log_prob(action).sum(dim=-1)
        entropy = dist.entropy().sum(dim=-1)
        value = self.critic(features)
        return action_log_prob, entropy, value


class PPOMemory:
    def __init__(self):
        self.states = []
        self.actions = []
        self.log_probs = []
        self.rewards = []
        self.values = []
        self.dones = []
    
    def store(self, state, action, log_prob, reward, value, done):
        self.states.append(state)
        self.actions.append(action)
        self.log_probs.append(log_prob)
        self.rewards.append(reward)
        self.values.append(value)
        self.dones.append(done)
    
    def clear(self):
        self.states = []
        self.actions = []
        self.log_probs = []
        self.rewards = []
        self.values = []
        self.dones = []
    
    def get(self):
        return (
            torch.FloatTensor(np.array(self.states)),
            torch.FloatTensor(np.array(self.actions)),
            torch.FloatTensor(np.array(self.log_probs)),
            torch.FloatTensor(np.array(self.rewards)),
            torch.FloatTensor(np.array(self.values)),
            torch.FloatTensor(np.array(self.dones))
        )


class CarlaLaneKeepingEnv:
    def __init__(self):
        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world('Town04')
        
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()
        self.vehicle = None
        self.sensors = []
        self.collision_hist = []
        self.spawn_points = self.map.get_spawn_points()
        
        self.distance_traveled = 0.0
        self.last_location = None
        self.steps = 0
        self.last_lane_offset = 0.0
        self.last_heading_error = 0.0
    
    def reset(self):
        self._cleanup()
        
        vehicle_bp = self.blueprint_library.filter('vehicle.tesla.model3')[0]
        spawn_point = np.random.choice(self.spawn_points[:30])
        
        try:
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        except:
            spawn_point = np.random.choice(self.spawn_points)
            self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        collision_sensor.listen(lambda e: self.collision_hist.append(e))
        self.sensors.append(collision_sensor)
        
        self.collision_hist.clear()
        self.last_location = self.vehicle.get_location()
        self.distance_traveled = 0.0
        self.steps = 0
        self.last_lane_offset = 0.0
        self.last_heading_error = 0.0
        
        for _ in range(10):
            self.world.tick()
        
        return self._get_state()
    
    def _get_state(self):
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        waypoint = self.map.get_waypoint(
            transform.location, project_to_road=True, lane_type=carla.LaneType.Driving
        )
        
        if not waypoint:
            return np.zeros(12, dtype=np.float32)
        
        lane_center = waypoint.transform.location
        offset_vec = transform.location - lane_center
        right_vec = transform.get_right_vector()
        lane_offset = offset_vec.x * right_vec.x + offset_vec.y * right_vec.y
        
        lane_yaw = waypoint.transform.rotation.yaw
        vehicle_yaw = transform.rotation.yaw
        heading_error = (vehicle_yaw - lane_yaw + 180) % 360 - 180
        heading_error_rad = np.radians(heading_error)
        
        next_wps = waypoint.next(10.0)
        if next_wps:
            next_wp = next_wps[0]
            wp_vec = next_wp.transform.location - transform.location
            forward_vec = transform.get_forward_vector()
            angle_to_wp = np.arctan2(
                wp_vec.y * forward_vec.x - wp_vec.x * forward_vec.y,
                wp_vec.x * forward_vec.x + wp_vec.y * forward_vec.y
            )
        else:
            angle_to_wp = 0.0
        
        curvature = 0.0
        if next_wps:
            curvature_yaw = next_wps[0].transform.rotation.yaw - lane_yaw
            curvature = np.sin(np.radians(curvature_yaw))
        
        speed_norm = speed / 15.0
        vel_x = velocity.x / 15.0
        vel_y = velocity.y / 15.0
        is_junction = 1.0 if waypoint.is_junction else 0.0
        
        state = np.array([
            lane_offset / 3.0,
            np.sin(heading_error_rad),
            np.cos(heading_error_rad),
            np.sin(angle_to_wp),
            np.cos(angle_to_wp),
            curvature,
            speed_norm,
            vel_x,
            vel_y,
            is_junction,
            lane_offset / 3.0 - self.last_lane_offset / 3.0,
            heading_error_rad - self.last_heading_error,
        ], dtype=np.float32)
        
        self.last_lane_offset = lane_offset
        self.last_heading_error = heading_error_rad
        
        return state
    
    def step(self, action):
        steering = float(np.clip(action[0], -0.4, 0.4))
        
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        speed_error = TARGET_SPEED - speed
        base_throttle = 0.5 + 0.1 * speed_error
        throttle_delta = float(np.clip(action[1], -0.3, 0.3))
        throttle = float(np.clip(base_throttle + throttle_delta, 0.0, 0.8))
        
        control = carla.VehicleControl(
            throttle=throttle,
            steer=steering,
            brake=0.0
        )
        
        self.vehicle.apply_control(control)
        self.world.tick()
        self.steps += 1
        
        current_location = self.vehicle.get_location()
        if self.last_location:
            self.distance_traveled += current_location.distance(self.last_location)
        self.last_location = current_location
        
        next_state = self._get_state()
        reward = self._calculate_reward(next_state, steering, throttle)
        done = self._is_done()
        
        return next_state, reward, done
    
    def _calculate_reward(self, state, steering, throttle):
        reward = 0.0
        
        lane_offset = state[0] * 3.0
        heading_sin = state[1]
        heading_cos = state[2]
        heading_error = np.arctan2(heading_sin, heading_cos)
        speed = state[6] * 15.0
        
        lane_reward = 5.0 * np.exp(-3.0 * abs(lane_offset))
        reward += lane_reward
        
        heading_reward = 3.0 * np.exp(-2.0 * abs(heading_error))
        reward += heading_reward
        
        reward += 0.1
        
        speed_error = abs(speed - TARGET_SPEED)
        speed_reward = 2.0 * np.exp(-speed_error / 3.0)
        reward += speed_reward
        
        steering_penalty = -0.5 * abs(steering)
        reward += steering_penalty
        
        if abs(lane_offset) > 1.0:
            reward -= 5.0
        
        if len(self.collision_hist) > 0:
            reward -= 100.0
        
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint is None or transform.location.distance(waypoint.transform.location) > 3.5:
            reward -= 50.0
        
        return reward
    
    def _is_done(self):
        if len(self.collision_hist) > 0:
            return True
        
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        if waypoint is None:
            return True
        if transform.location.distance(waypoint.transform.location) > 3.5:
            return True
        
        if self.steps >= MAX_STEPS:
            return True
        
        return False
    
    def _cleanup(self):
        if self.vehicle:
            self.vehicle.destroy()
        for sensor in self.sensors:
            if sensor.is_alive:
                sensor.destroy()
        self.sensors.clear()
        self.vehicle = None
    
    def close(self):
        self._cleanup()
        settings = self.world.get_settings()
        settings.synchronous_mode = False
        self.world.apply_settings(settings)


class PPOAgent:
    def __init__(self, state_dim=12, action_dim=2):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.policy = ActorCritic(state_dim, action_dim).to(self.device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=LEARNING_RATE)
        self.memory = PPOMemory()
    
    def select_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            action, log_prob, value = self.policy.act(state_tensor)
        
        return action.cpu().numpy()[0], log_prob.cpu().item(), value.cpu().item()
    
    def update(self):
        states, actions, old_log_probs, rewards, values, dones = self.memory.get()
        
        states = states.to(self.device)
        actions = actions.to(self.device)
        old_log_probs = old_log_probs.to(self.device)
        
        advantages = []
        returns = []
        
        gae = 0
        next_value = 0
        
        for t in reversed(range(len(rewards))):
            if t == len(rewards) - 1:
                next_value = 0
                next_done = 0
            else:
                next_value = values[t + 1]
                next_done = dones[t + 1]
            
            delta = rewards[t] + GAMMA * next_value * (1 - next_done) - values[t]
            gae = delta + GAMMA * GAE_LAMBDA * (1 - next_done) * gae
            
            advantages.insert(0, gae)
            returns.insert(0, gae + values[t])
        
        advantages = torch.FloatTensor(advantages).to(self.device)
        returns = torch.FloatTensor(returns).to(self.device)
        
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        total_policy_loss = 0
        total_value_loss = 0
        total_entropy = 0
        
        for _ in range(UPDATE_EPOCHS):
            log_probs, entropy, state_values = self.policy.evaluate(states, actions)
            
            ratios = torch.exp(log_probs - old_log_probs)
            
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - CLIP_EPSILON, 1 + CLIP_EPSILON) * advantages
            
            policy_loss = -torch.min(surr1, surr2).mean()
            value_loss = 0.5 * ((returns - state_values.squeeze()) ** 2).mean()
            entropy_loss = -entropy.mean()
            
            loss = policy_loss + VALUE_COEF * value_loss + ENTROPY_COEF * entropy_loss
            
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.policy.parameters(), 0.5)
            self.optimizer.step()
            
            total_policy_loss += policy_loss.item()
            total_value_loss += value_loss.item()
            total_entropy += entropy_loss.item()
        
        self.memory.clear()
        
        return {
            'policy_loss': total_policy_loss / UPDATE_EPOCHS,
            'value_loss': total_value_loss / UPDATE_EPOCHS,
            'entropy': -total_entropy / UPDATE_EPOCHS
        }


# ==================== CONTINUE TRAINING ====================
def continue_training():
    print("="*80)
    print("CONTINUING PPO TRAINING FROM CHECKPOINT")
    print("="*80)
    print(f"Checkpoint: {CHECKPOINT_PATH}")
    print(f"Additional episodes: {EPISODES}")
    print(f"Starting from episode: {START_EPISODE}\n")
    
    env = CarlaLaneKeepingEnv()
    agent = PPOAgent(state_dim=12, action_dim=2)
    
    # LOAD CHECKPOINT
    try:
        agent.policy.load_state_dict(torch.load(CHECKPOINT_PATH))
        print(f"✓ Checkpoint loaded successfully!")
        print(f"✓ Continuing training from episode {START_EPISODE}\n")
    except Exception as e:
        print(f"✗ Failed to load checkpoint: {e}")
        return
    
    episode_rewards = []
    episode_distances = []
    episode_lengths = []
    policy_losses = []
    value_losses = []
    
    plt.ion()
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    
    best_distance = 0.0
    
    print("Resuming training...\n")
    start_time = time.time()
    
    for episode in range(EPISODES):
        state = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            action, log_prob, value = agent.select_action(state)
            next_state, reward, done = env.step(action)
            agent.memory.store(state, action, log_prob, reward, value, done)
            
            episode_reward += reward
            state = next_state
        
        losses = agent.update()
        
        episode_rewards.append(episode_reward)
        episode_distances.append(env.distance_traveled)
        episode_lengths.append(env.steps)
        policy_losses.append(losses['policy_loss'])
        value_losses.append(losses['value_loss'])
        
        if (episode + 1) % 10 == 0:
            avg_reward = np.mean(episode_rewards[-10:])
            avg_distance = np.mean(episode_distances[-10:])
            avg_length = np.mean(episode_lengths[-10:])
            elapsed = (time.time() - start_time) / 60
            
            total_ep = START_EPISODE + episode + 1
            
            print(f"Ep {total_ep:3d} | "
                  f"R: {episode_reward:7.1f} | "
                  f"D: {env.distance_traveled:6.1f}m | "
                  f"L: {env.steps:3d} | "
                  f"Avg10: R={avg_reward:6.1f}, D={avg_distance:5.1f}m | "
                  f"Loss: P={losses['policy_loss']:.3f}, V={losses['value_loss']:.3f} | "
                  f"T: {elapsed:.1f}min")
            
            if avg_distance > best_distance:
                best_distance = avg_distance
                torch.save(agent.policy.state_dict(), 'ppo_lane_keeping_best_continued.pth')
                print(f"  ✓ New best! Avg distance: {avg_distance:.1f}m")
        
        if (episode + 1) % 5 == 0:
            ax1.clear()
            ax1.plot(episode_rewards, alpha=0.3)
            ax1.plot(np.convolve(episode_rewards, np.ones(10)/10, mode='valid'))
            ax1.set_title('Episode Rewards (Continued Training)')
            ax1.set_xlabel('Episode')
            ax1.set_ylabel('Reward')
            ax1.grid(True)
            
            ax2.clear()
            ax2.plot(episode_distances, alpha=0.3)
            ax2.plot(np.convolve(episode_distances, np.ones(10)/10, mode='valid'))
            ax2.set_title('Distance Traveled (Continued Training)')
            ax2.set_xlabel('Episode')
            ax2.set_ylabel('Distance (m)')
            ax2.grid(True)
            
            ax3.clear()
            ax3.plot(policy_losses, label='Policy Loss')
            ax3.plot(value_losses, label='Value Loss')
            ax3.set_title('Training Losses')
            ax3.set_xlabel('Episode')
            ax3.set_ylabel('Loss')
            ax3.legend()
            ax3.grid(True)
            
            ax4.clear()
            ax4.plot(episode_lengths, alpha=0.3)
            ax4.plot(np.convolve(episode_lengths, np.ones(10)/10, mode='valid'))
            ax4.set_title('Episode Length')
            ax4.set_xlabel('Episode')
            ax4.set_ylabel('Steps')
            ax4.grid(True)
            
            plt.tight_layout()
            plt.pause(0.01)
            plt.savefig(f'ppo_continued_training_ep{START_EPISODE + episode + 1}.png', dpi=150)
        
        if (episode + 1) % 50 == 0:
            torch.save(agent.policy.state_dict(), f'ppo_checkpoint_ep{START_EPISODE + episode + 1}.pth')
    
    torch.save(agent.policy.state_dict(), 'ppo_lane_keeping_final_continued.pth')
    
    print(f"\n✓ Continued training completed in {(time.time() - start_time) / 60:.1f} minutes")
    print(f"✓ Total episodes: {START_EPISODE + EPISODES}")
    print(f"✓ Best avg distance: {best_distance:.1f}m")
    print(f"✓ Model saved as: ppo_lane_keeping_best_continued.pth")
    
    plt.ioff()
    plt.show()
    
    env.close()


if __name__ == "__main__":
    continue_training()

================================================================================
CONTINUING PPO TRAINING FROM CHECKPOINT
================================================================================
Checkpoint: ppo_lane_keeping_best.pth
Additional episodes: 500
Starting from episode: 500

carla_rl_training.py:2388: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent.policy.load_state_dict(torch.load(CHECKPOINT_PATH))
✓ Checkpoint loaded successfully!
✓ Continuing training from episode 500

Resuming training...

Ep 510 | R:  1456.8 | D:  456.6m | L: 283 | Avg10: R=4601.4, D=599.5m | Loss: P=-0.001, V=7041.839 | T: 0.6min
  ✓ New best! Avg distance: 599.5m
Ep 520 | R:  5543.1 | D:  638.1m | L: 800 | Avg10: R=5170.4, D=608.1m | Loss: P=0.012, V=2285.150 | T: 1.3min
  ✓ New best! Avg distance: 608.1m
Ep 530 | R:  5154.8 | D:  629.6m | L: 800 | Avg10: R=4953.8, D=621.0m | Loss: P=0.017, V=1702.724 | T: 2.1min
  ✓ New best! Avg distance: 621.0m
Ep 540 | R:  4986.0 | D:  651.1m | L: 800 | Avg10: R=4783.8, D=602.5m | Loss: P=-0.001, V=1583.608 | T: 2.8min
Ep 550 | R:  5136.3 | D:  652.7m | L: 800 | Avg10: R=5148.2, D=622.4m | Loss: P=-0.001, V=1798.660 | T: 3.6min
  ✓ New best! Avg distance: 622.4m
Ep 560 | R:  5162.8 | D:  595.5m | L: 800 | Avg10: R=4976.1, D=627.9m | Loss: P=-0.001, V=1783.139 | T: 4.4min
  ✓ New best! Avg distance: 627.9m
Ep 570 | R:  5250.6 | D:  630.1m | L: 800 | Avg10: R=5067.0, D=622.1m | Loss: P=0.002, V=1770.697 | T: 5.2min
Ep 580 | R:  5347.6 | D:  590.8m | L: 800 | Avg10: R=5223.7, D=605.6m | Loss: P=0.000, V=1984.345 | T: 6.0min
Ep 590 | R:  4985.4 | D:  637.7m | L: 800 | Avg10: R=5282.2, D=612.9m | Loss: P=-0.004, V=1823.793 | T: 6.8min
Ep 600 | R:  5266.9 | D:  598.4m | L: 800 | Avg10: R=5186.7, D=620.6m | Loss: P=-0.005, V=1817.128 | T: 7.6min
Ep 610 | R:  5535.6 | D:  631.2m | L: 800 | Avg10: R=5313.3, D=601.1m | Loss: P=-0.002, V=1963.984 | T: 8.4min
Ep 620 | R:  5102.7 | D:  595.5m | L: 800 | Avg10: R=5344.0, D=611.1m | Loss: P=-0.006, V=1786.040 | T: 9.1min
Ep 630 | R:  5059.5 | D:  594.9m | L: 800 | Avg10: R=5017.7, D=602.1m | Loss: P=-0.002, V=2378.541 | T: 9.9min
Ep 640 | R:  5363.6 | D:  591.1m | L: 800 | Avg10: R=5204.6, D=617.6m | Loss: P=-0.002, V=1868.886 | T: 10.7min
Ep 650 | R:  4910.7 | D:  591.2m | L: 800 | Avg10: R=5217.7, D=621.0m | Loss: P=-0.002, V=3035.562 | T: 11.5min
Ep 660 | R:  4368.0 | D:  651.4m | L: 800 | Avg10: R=4659.3, D=601.1m | Loss: P=0.000, V=1838.808 | T: 12.1min
Ep 670 | R:  4682.1 | D:  657.8m | L: 800 | Avg10: R=4949.1, D=622.1m | Loss: P=-0.002, V=1373.904 | T: 12.9min
Ep 680 | R:  5211.6 | D:  619.0m | L: 800 | Avg10: R=5295.1, D=616.7m | Loss: P=-0.003, V=1769.187 | T: 13.6min
Ep 690 | R:  3823.3 | D:  633.2m | L: 800 | Avg10: R=2831.5, D=636.7m | Loss: P=-0.004, V=1023.465 | T: 14.4min
  ✓ New best! Avg distance: 636.7m
Ep 700 | R:  4212.7 | D:  629.9m | L: 800 | Avg10: R=4096.1, D=609.5m | Loss: P=0.000, V=1164.180 | T: 15.2min
Ep 710 | R:  5584.6 | D:  658.5m | L: 800 | Avg10: R=4870.5, D=599.5m | Loss: P=-0.001, V=1805.789 | T: 15.9min
Ep 720 | R:  5058.9 | D:  650.5m | L: 800 | Avg10: R=5204.3, D=621.1m | Loss: P=0.006, V=1719.087 | T: 16.7min
Ep 730 | R:  5250.6 | D:  638.5m | L: 800 | Avg10: R=4927.2, D=609.0m | Loss: P=-0.001, V=1801.079 | T: 17.5min
Ep 740 | R:  4960.2 | D:  639.5m | L: 800 | Avg10: R=5011.1, D=623.3m | Loss: P=-0.001, V=1907.849 | T: 18.3min
Ep 750 | R:  5153.4 | D:  614.7m | L: 800 | Avg10: R=4719.4, D=606.0m | Loss: P=-0.000, V=1797.341 | T: 18.9min
Ep 760 | R:  5389.2 | D:  639.6m | L: 800 | Avg10: R=5315.8, D=634.0m | Loss: P=-0.001, V=2109.483 | T: 19.7min
Ep 770 | R:  3969.1 | D:  634.6m | L: 800 | Avg10: R=4288.1, D=624.9m | Loss: P=0.003, V=1248.874 | T: 20.5min
Ep 780 | R:  5172.2 | D:  652.2m | L: 800 | Avg10: R=4796.4, D=619.0m | Loss: P=-0.003, V=1563.332 | T: 21.3min
Ep 790 | R:  4835.4 | D:  629.8m | L: 800 | Avg10: R=5322.2, D=623.9m | Loss: P=-0.000, V=1576.500 | T: 22.1min
Ep 800 | R:  4840.2 | D:  601.6m | L: 800 | Avg10: R=5148.2, D=628.1m | Loss: P=0.004, V=2511.006 | T: 22.8min
Ep 810 | R:  5183.0 | D:  613.0m | L: 800 | Avg10: R=4848.7, D=609.3m | Loss: P=-0.002, V=2119.789 | T: 23.6min
Ep 820 | R:  5352.9 | D:  591.7m | L: 800 | Avg10: R=4845.7, D=597.9m | Loss: P=-0.002, V=1784.175 | T: 24.3min
Ep 830 | R:  5063.5 | D:  629.9m | L: 800 | Avg10: R=5256.9, D=616.0m | Loss: P=-0.001, V=1895.083 | T: 25.1min
Ep 840 | R:  5310.1 | D:  598.0m | L: 800 | Avg10: R=5034.1, D=624.5m | Loss: P=-0.002, V=1854.212 | T: 25.8min
Ep 850 | R:  5035.0 | D:  629.9m | L: 800 | Avg10: R=5262.4, D=608.9m | Loss: P=-0.005, V=1445.483 | T: 26.6min
Ep 860 | R:  5365.6 | D:  621.0m | L: 800 | Avg10: R=5151.3, D=611.6m | Loss: P=-0.001, V=2036.254 | T: 27.4min
Ep 870 | R:  5009.2 | D:  658.8m | L: 800 | Avg10: R=5231.5, D=634.0m | Loss: P=-0.003, V=1751.550 | T: 28.1min
Ep 880 | R:  5497.9 | D:  636.5m | L: 800 | Avg10: R=5176.1, D=610.1m | Loss: P=-0.002, V=1758.230 | T: 28.8min
Ep 890 | R:  5408.5 | D:  614.0m | L: 800 | Avg10: R=5021.2, D=607.2m | Loss: P=-0.000, V=2300.044 | T: 29.6min
Ep 900 | R:  5365.1 | D:  598.6m | L: 800 | Avg10: R=4999.7, D=601.1m | Loss: P=-0.000, V=1828.457 | T: 30.3min
Ep 910 | R:  5436.8 | D:  597.7m | L: 800 | Avg10: R=5256.2, D=618.1m | Loss: P=-0.001, V=1942.966 | T: 31.1min
Ep 920 | R:  4465.2 | D:  600.5m | L: 800 | Avg10: R=5157.5, D=619.1m | Loss: P=0.011, V=2764.228 | T: 31.9min
Ep 930 | R:  5137.1 | D:  595.4m | L: 800 | Avg10: R=5089.9, D=617.6m | Loss: P=0.001, V=2434.797 | T: 32.8min
Ep 940 | R:  5350.8 | D:  581.1m | L: 800 | Avg10: R=5364.5, D=621.7m | Loss: P=-0.004, V=2088.951 | T: 33.5min
Ep 950 | R:  5141.1 | D:  614.5m | L: 800 | Avg10: R=5060.9, D=620.6m | Loss: P=-0.001, V=1833.919 | T: 34.2min
Ep 960 | R:  5579.9 | D:  631.8m | L: 800 | Avg10: R=5386.3, D=631.4m | Loss: P=-0.000, V=1842.848 | T: 35.0min
Ep 970 | R:  5516.2 | D:  583.9m | L: 800 | Avg10: R=5341.5, D=617.5m | Loss: P=-0.001, V=1948.251 | T: 35.8min
Ep 980 | R:  5460.4 | D:  620.4m | L: 800 | Avg10: R=5239.5, D=611.4m | Loss: P=-0.001, V=2147.317 | T: 36.5min
Ep 990 | R:  4674.9 | D:  586.7m | L: 800 | Avg10: R=5298.5, D=610.5m | Loss: P=-0.003, V=2513.180 | T: 37.2min
Ep 1000 | R:  5081.4 | D:  622.3m | L: 800 | Avg10: R=5364.5, D=620.7m | Loss: P=-0.001, V=2526.472 | T: 38.0min

✓ Continued training completed in 38.0 minutes
✓ Total episodes: 1000
✓ Best avg distance: 636.7m
✓ Model saved as: ppo_lane_keeping_best_continued.pth

## evaulation

In [ ]:
"""
PPO Lane Keeping - Evaluation Script
Evaluates trained PPO model with video recording
"""

import carla
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import Normal
import time
import argparse
import os
import cv2
import imageio.v2 as imageio
from datetime import datetime

TARGET_SPEED = 7.0

# ==================== ACTOR-CRITIC (same as training) ====================
class ActorCritic(nn.Module):
    def __init__(self, state_dim=12, action_dim=2):
        super(ActorCritic, self).__init__()
        
        self.shared = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )
        
        self.actor_mean = nn.Linear(256, action_dim)
        self.actor_log_std = nn.Parameter(torch.zeros(action_dim))
        
        self.critic = nn.Linear(256, 1)
    
    def forward(self, state):
        features = self.shared(state)
        return features
    
    def act(self, state, deterministic=False):
        features = self.forward(state)
        
        action_mean = self.actor_mean(features)
        
        if deterministic:
            return action_mean
        else:
            action_std = torch.exp(self.actor_log_std)
            dist = Normal(action_mean, action_std)
            action = dist.sample()
            return action


# ==================== EVALUATION ENVIRONMENT ====================
class EvalEnv:
    def __init__(self, visualize=True):
        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world('Town04')
        
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)
        
        self.blueprint_library = self.world.get_blueprint_library()
        self.map = self.world.get_map()
        self.vehicle = None
        self.sensors = []
        self.camera = None
        self.visualize = visualize
        
        self.collision_hist = []
        self.spawn_points = self.map.get_spawn_points()
        
        self.distance_traveled = 0.0
        self.last_location = None
        self.steps = 0
        self.speeds = []
        self.lane_offsets = []
        
        self.recording_frames = []
        
        self.last_lane_offset = 0.0
        self.last_heading_error = 0.0
    
    def camera_callback(self, image):
        try:
            if image is None or not hasattr(image, 'raw_data'):
                return
            if not self.vehicle or not self.vehicle.is_alive:
                return
            
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            
            overlay = self._create_overlay(arr.copy())
            if overlay is not None:
                self.recording_frames.append(overlay)
        except:
            pass
    
    def _create_overlay(self, frame):
        try:
            if frame is None or frame.size == 0:
                return None
            
            overlay = frame.copy()
            cv2.rectangle(overlay, (10, 10), (450, 180), (0, 0, 0), -1)
            overlay = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)
            
            if self.vehicle and self.vehicle.is_alive:
                velocity = self.vehicle.get_velocity()
                speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
                
                cv2.putText(overlay, f"Step: {self.steps}", (20, 40), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(overlay, f"Speed: {speed:.1f} m/s ({speed*3.6:.1f} km/h)", (20, 70), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.putText(overlay, f"Distance: {self.distance_traveled:.1f} m", (20, 100), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                
                if self.lane_offsets:
                    lane_off = self.lane_offsets[-1]
                    color = (0, 255, 0) if abs(lane_off) < 0.3 else (0, 165, 255)
                    cv2.putText(overlay, f"Lane: {lane_off:+.3f} m", (20, 130), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                cv2.putText(overlay, "PPO AGENT", (20, 160), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
            
            return overlay
        except:
            return frame
    
    def reset(self, spawn_idx=0):
        self._cleanup()
        
        vehicle_bp = self.blueprint_library.filter('vehicle.tesla.model3')[0]
        spawn_point = self.spawn_points[spawn_idx % len(self.spawn_points)]
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        
        # Collision sensor
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        collision_sensor.listen(lambda e: self.collision_hist.append(e))
        self.sensors.append(collision_sensor)
        
        # Camera
        cam_bp = self.blueprint_library.find('sensor.camera.rgb')
        cam_bp.set_attribute('image_size_x', '1280')
        cam_bp.set_attribute('image_size_y', '720')
        cam_bp.set_attribute('fov', '90')
        cam_transform = carla.Transform(
            carla.Location(x=-6.0, z=2.8),
            carla.Rotation(pitch=-12)
        )
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)
        self.sensors.append(self.camera)
        
        self.collision_hist.clear()
        self.distance_traveled = 0.0
        self.last_location = self.vehicle.get_location()
        self.steps = 0
        self.speeds = []
        self.lane_offsets = []
        self.recording_frames = []
        
        self.last_lane_offset = 0.0
        self.last_heading_error = 0.0
        
        if self.visualize:
            spectator = self.world.get_spectator()
            t = self.vehicle.get_transform()
            spectator.set_transform(carla.Transform(
                t.location + carla.Location(x=-10, z=5),
                carla.Rotation(pitch=-20, yaw=t.rotation.yaw)
            ))
        
        for _ in range(10):
            self.world.tick()
        
        return self._get_state()
    
    def _get_state(self):
        """Same 12-dim state as training"""
        transform = self.vehicle.get_transform()
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        self.speeds.append(speed)
        
        waypoint = self.map.get_waypoint(
            transform.location, project_to_road=True, lane_type=carla.LaneType.Driving
        )
        
        if not waypoint:
            return np.zeros(12, dtype=np.float32)
        
        lane_center = waypoint.transform.location
        offset_vec = transform.location - lane_center
        right_vec = transform.get_right_vector()
        lane_offset = offset_vec.x * right_vec.x + offset_vec.y * right_vec.y
        
        self.lane_offsets.append(abs(lane_offset))
        
        lane_yaw = waypoint.transform.rotation.yaw
        vehicle_yaw = transform.rotation.yaw
        heading_error = (vehicle_yaw - lane_yaw + 180) % 360 - 180
        heading_error_rad = np.radians(heading_error)
        
        next_wps = waypoint.next(10.0)
        if next_wps:
            next_wp = next_wps[0]
            wp_vec = next_wp.transform.location - transform.location
            forward_vec = transform.get_forward_vector()
            angle_to_wp = np.arctan2(
                wp_vec.y * forward_vec.x - wp_vec.x * forward_vec.y,
                wp_vec.x * forward_vec.x + wp_vec.y * forward_vec.y
            )
        else:
            angle_to_wp = 0.0
        
        curvature = 0.0
        if next_wps:
            curvature_yaw = next_wps[0].transform.rotation.yaw - lane_yaw
            curvature = np.sin(np.radians(curvature_yaw))
        
        speed_norm = speed / 15.0
        vel_x = velocity.x / 15.0
        vel_y = velocity.y / 15.0
        is_junction = 1.0 if waypoint.is_junction else 0.0
        
        state = np.array([
            lane_offset / 3.0,
            np.sin(heading_error_rad),
            np.cos(heading_error_rad),
            np.sin(angle_to_wp),
            np.cos(angle_to_wp),
            curvature,
            speed_norm,
            vel_x,
            vel_y,
            is_junction,
            lane_offset / 3.0 - self.last_lane_offset / 3.0,
            heading_error_rad - self.last_heading_error,
        ], dtype=np.float32)
        
        self.last_lane_offset = lane_offset
        self.last_heading_error = heading_error_rad
        
        return state
    
    def step(self, action):
        steering = float(np.clip(action[0], -0.4, 0.4))
        
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        speed_error = TARGET_SPEED - speed
        base_throttle = 0.5 + 0.1 * speed_error
        throttle_delta = float(np.clip(action[1], -0.3, 0.3))
        throttle = float(np.clip(base_throttle + throttle_delta, 0.0, 0.8))
        
        control = carla.VehicleControl(
            throttle=throttle,
            steer=steering,
            brake=0.0
        )
        
        self.vehicle.apply_control(control)
        
        if self.visualize:
            try:
                spectator = self.world.get_spectator()
                t = self.vehicle.get_transform()
                spec_location = t.location - t.get_forward_vector() * 10 + carla.Location(z=5)
                spectator.set_transform(carla.Transform(
                    spec_location,
                    carla.Rotation(pitch=-20, yaw=t.rotation.yaw)
                ))
            except:
                pass
        
        self.world.tick()
        self.steps += 1
        
        current_location = self.vehicle.get_location()
        if self.last_location:
            self.distance_traveled += current_location.distance(self.last_location)
        self.last_location = current_location
        
        next_state = self._get_state()
        done = self._is_done()
        
        return next_state, done
    
    def _is_done(self):
        if len(self.collision_hist) > 0:
            print(f"\n  ⚠️  COLLISION at step {self.steps}")
            return True
        
        transform = self.vehicle.get_transform()
        waypoint = self.map.get_waypoint(transform.location, project_to_road=True)
        
        if waypoint is None:
            print(f"\n  ⚠️  OFF ROAD")
            return True
        
        if transform.location.distance(waypoint.transform.location) > 3.5:
            print(f"\n  ⚠️  TOO FAR FROM LANE")
            return True
        
        return False
    
    def save_video(self, episode_num, output_dir="ppo_eval_videos"):
        if not self.recording_frames:
            return None
        
        valid_frames = [f for f in self.recording_frames if f is not None and f.size > 0]
        if not valid_frames:
            return None
        
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        video_path = os.path.join(output_dir, f"ppo_ep{episode_num}_{timestamp}.mp4")
        
        try:
            print(f"  Saving video ({len(valid_frames)} frames)...")
            with imageio.get_writer(video_path, fps=20, codec='libx264', quality=8, pixelformat='yuv420p') as writer:
                for frame in valid_frames:
                    try:
                        writer.append_data(frame)
                    except:
                        continue
            print(f"  ✓ Video: {video_path}")
            return video_path
        except Exception as e:
            print(f"  ✗ Video failed: {e}")
            return None
    
    def _cleanup(self):
        if self.camera and self.camera.is_alive:
            try:
                self.camera.stop()
                time.sleep(0.1)
            except:
                pass
        if self.vehicle and self.vehicle.is_alive:
            try:
                self.vehicle.destroy()
            except:
                pass
        for sensor in self.sensors:
            try:
                if sensor.is_alive:
                    sensor.destroy()
            except:
                pass
        self.sensors.clear()
        self.vehicle = None
        self.camera = None
    
    def close(self):
        self._cleanup()
        settings = self.world.get_settings()
        settings.synchronous_mode = False
        self.world.apply_settings(settings)


# ==================== EVALUATION ====================
def evaluate(model_path, num_episodes=5, max_steps=2000, deterministic=True):
    print("="*80)
    print("PPO LANE KEEPING - EVALUATION")
    print("="*80)
    print(f"Model: {model_path}")
    print(f"Episodes: {num_episodes}")
    print(f"Deterministic: {deterministic}\n")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    policy = ActorCritic(state_dim=12, action_dim=2).to(device)
    
    try:
        policy.load_state_dict(torch.load(model_path, map_location=device))
        policy.eval()
        print(f"✓ Model loaded\n")
    except Exception as e:
        print(f"✗ Failed to load: {e}")
        return
    
    env = EvalEnv(visualize=True)
    
    all_distances = []
    all_speeds = []
    all_lane_offsets = []
    all_collisions = 0
    video_paths = []
    
    for ep in range(num_episodes):
        print(f"\n{'='*80}")
        print(f"EPISODE {ep+1}/{num_episodes}")
        print(f"{'='*80}\n")
        
        state = env.reset(spawn_idx=ep * 15)
        done = False
        steps = 0
        
        while not done and steps < max_steps:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            
            with torch.no_grad():
                action = policy.act(state_tensor, deterministic=deterministic)
            
            next_state, done = env.step(action.cpu().numpy()[0])
            state = next_state
            steps += 1
            
            if steps % 200 == 0:
                avg_speed = np.mean(env.speeds[-100:]) if len(env.speeds) >= 100 else np.mean(env.speeds)
                avg_lane = np.mean(env.lane_offsets[-100:]) if len(env.lane_offsets) >= 100 else np.mean(env.lane_offsets)
                print(f"  Step {steps:4d}: Speed={avg_speed:.1f} m/s | Dist={env.distance_traveled:.1f}m | Lane={avg_lane:.3f}m")
        
        # Summary
        print(f"\n{'─'*80}")
        print(f"EPISODE {ep+1} SUMMARY")
        print(f"{'─'*80}\n")
        
        outcome = "✗ FAILED" if len(env.collision_hist) > 0 else "✓ SUCCESS"
        print(f"Outcome:     {outcome}")
        print(f"Distance:    {env.distance_traveled:.1f} m")
        print(f"Steps:       {steps}/{max_steps}")
        print(f"Avg Speed:   {np.mean(env.speeds):.1f} m/s ({np.mean(env.speeds)*3.6:.1f} km/h)")
        print(f"Avg Lane:    {np.mean(env.lane_offsets):.3f} m")
        print(f"Max Lane:    {max(env.lane_offsets) if env.lane_offsets else 0:.3f} m\n")
        
        all_distances.append(env.distance_traveled)
        all_speeds.append(np.mean(env.speeds))
        all_lane_offsets.append(np.mean(env.lane_offsets))
        if len(env.collision_hist) > 0:
            all_collisions += 1
        
        video_path = env.save_video(ep + 1)
        if video_path:
            video_paths.append(video_path)
        
        print()
        time.sleep(1)
    
    # Overall results
    print(f"\n{'='*80}")
    print("OVERALL RESULTS")
    print(f"{'='*80}\n")
    
    success_rate = ((num_episodes - all_collisions) / num_episodes) * 100
    print(f"Success Rate:       {success_rate:.1f}%")
    print(f"Avg Distance:       {np.mean(all_distances):.1f} m")
    print(f"Avg Speed:          {np.mean(all_speeds):.1f} m/s ({np.mean(all_speeds)*3.6:.1f} km/h)")
    print(f"Avg Lane Offset:    {np.mean(all_lane_offsets):.3f} m\n")
    
    # Scoring
    speed_score = min(100, (np.mean(all_speeds) / 7.0) * 100)
    safety_score = success_rate
    distance_score = min(100, (np.mean(all_distances) / 1000.0) * 100)
    lane_score = max(0, 100 - (np.mean(all_lane_offsets) / 0.5) * 100)
    
    overall_score = (speed_score + safety_score + distance_score + lane_score) / 4
    
    print(f"{'='*80}")
    print("SCORING")
    print(f"{'='*80}\n")
    print(f"Speed:          {speed_score:.1f}%")
    print(f"Safety:         {safety_score:.1f}%")
    print(f"Distance:       {distance_score:.1f}%")
    print(f"Lane Keeping:   {lane_score:.1f}%")
    print(f"OVERALL:        {overall_score:.1f}%\n")
    
    if overall_score >= 85:
        grade = "A - Excellent"
    elif overall_score >= 75:
        grade = "B - Good"
    elif overall_score >= 65:
        grade = "C - Fair"
    else:
        grade = "D - Needs Improvement"
    
    print(f"GRADE: {grade}\n")
    
    print("VIDEOS:")
    for vp in video_paths:
        print(f"  • {vp}")
    
    print(f"\n{'='*80}\n")
    
    env.close()


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', type=str, default='ppo_lane_keeping_best.pth')
    parser.add_argument('--episodes', type=int, default=5)
    parser.add_argument('--max_steps', type=int, default=2000)
    parser.add_argument('--stochastic', action='store_true', help='Use stochastic policy')
    
    args = parser.parse_args()
    
    print("\n⚠️  Make sure CARLA is running!\n")
    input("Press Enter to start...")
    
    evaluate(args.model, args.episodes, args.max_steps, deterministic=not args.stochastic)

    #VIDEOS: ppo_eval_videos/ppo_ep1_20260103_194607.mp4 inside PPO_best_git folder




================================================================================

EPISODE 1/1

================================================================================

  Step  200: Speed=5.1 m/s | Dist=467.1m | Lane=0.145m

  Step  400: Speed=5.1 m/s | Dist=518.0m | Lane=0.172m

  Step  600: Speed=5.1 m/s | Dist=568.9m | Lane=0.143m

  Step  800: Speed=5.1 m/s | Dist=619.8m | Lane=0.101m

  Step 1000: Speed=5.1 m/s | Dist=670.7m | Lane=0.042m

  Step 1200: Speed=5.1 m/s | Dist=721.5m | Lane=0.076m

  Step 1400: Speed=5.0 m/s | Dist=771.9m | Lane=0.238m

  Step 1600: Speed=5.1 m/s | Dist=822.3m | Lane=0.236m

  Step 1800: Speed=5.0 m/s | Dist=872.7m | Lane=0.317m

  Step 2000: Speed=5.0 m/s | Dist=922.9m | Lane=0.442m

────────────────────────────────────────────────────────────────────────────────

EPISODE 1 SUMMARY

────────────────────────────────────────────────────────────────────────────────

Outcome:     ✓ SUCCESS

Distance:    922.9 m

Steps:       2000/2000

Avg Speed:   5.0 m/s (17.8 km/h)

Avg Lane:    0.175 m

Max Lane:    1.667 m

  Saving video (1462 frames)...

  ✓ Video: ppo_eval_videos/ppo_ep1_20260103_194607.mp4

================================================================================

OVERALL RESULTS

================================================================================

Success Rate:       100.0%

Avg Distance:       922.9 m

Avg Speed:          5.0 m/s (17.8 km/h)

Avg Lane Offset:    0.175 m

================================================================================

SCORING

================================================================================

Speed:          70.8%

Safety:         100.0%

Distance:       92.3%

Lane Keeping:   65.0%

OVERALL:        82.0%

GRADE: B - Good

VIDEOS:

  • ppo_eval_videos/ppo_ep1_20260103_194607.mp4

================================================================================

 python3 carla_evaluation.py  --model /home/robotuser/carla_0.9.12/PythonAPI/examples/ppo_lane_keeping_final_continued.pth --episodes 1 --max_steps 2500

6:14:17 (carla_env) robotuser@kuorobot02 examples → python3 carla_evaluation.py  --model /home/robotuser/carla_0.9.12/PythonAPI/examples/ppo_lane_keeping_final_continued.pth --episodes 2 --max_steps 2000

⚠️  Make sure CARLA is running!

Press Enter to start...
================================================================================
PPO LANE KEEPING - EVALUATION
================================================================================
Model: /home/robotuser/carla_0.9.12/PythonAPI/examples/ppo_lane_keeping_final_continued.pth
Episodes: 2
Deterministic: True

carla_evaluation.py:3226: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy.load_state_dict(torch.load(model_path, map_location=device))
✓ Model loaded


================================================================================
EPISODE 1/2
================================================================================

  Step  200: Speed=5.1 m/s | Dist=467.0m | Lane=0.049m
  Step  400: Speed=5.1 m/s | Dist=517.6m | Lane=0.114m
  Step  600: Speed=5.1 m/s | Dist=568.2m | Lane=0.161m
  Step  800: Speed=5.1 m/s | Dist=618.8m | Lane=0.169m
  Step 1000: Speed=5.1 m/s | Dist=669.4m | Lane=0.136m
  Step 1200: Speed=5.1 m/s | Dist=720.0m | Lane=0.222m
  Step 1400: Speed=5.1 m/s | Dist=770.6m | Lane=0.171m
  Step 1600: Speed=5.1 m/s | Dist=821.2m | Lane=0.140m
  Step 1800: Speed=5.2 m/s | Dist=872.3m | Lane=0.425m
  Step 2000: Speed=5.1 m/s | Dist=923.1m | Lane=0.112m

────────────────────────────────────────────────────────────────────────────────
EPISODE 1 SUMMARY
────────────────────────────────────────────────────────────────────────────────

Outcome:     ✓ SUCCESS
Distance:    923.1 m
Steps:       2000/2000
Avg Speed:   5.0 m/s (17.8 km/h)
Avg Lane:    0.146 m
Max Lane:    1.351 m

  Saving video (1323 frames)...
  ✓ Video: ppo_eval_videos/ppo_ep1_20260108_181515.mp4


================================================================================
EPISODE 2/2
================================================================================

  Step  200: Speed=5.1 m/s | Dist=467.8m | Lane=0.096m
  Step  400: Speed=5.1 m/s | Dist=518.4m | Lane=0.103m
  Step  600: Speed=5.1 m/s | Dist=569.0m | Lane=0.095m
  Step  800: Speed=5.1 m/s | Dist=619.6m | Lane=0.061m
  Step 1000: Speed=5.0 m/s | Dist=670.0m | Lane=0.047m
  Step 1200: Speed=5.0 m/s | Dist=720.1m | Lane=0.012m
  Step 1400: Speed=4.9 m/s | Dist=769.1m | Lane=0.020m
  Step 1600: Speed=4.9 m/s | Dist=817.9m | Lane=0.018m
  Step 1800: Speed=5.0 m/s | Dist=867.9m | Lane=0.012m
  Step 2000: Speed=5.1 m/s | Dist=918.3m | Lane=0.237m

────────────────────────────────────────────────────────────────────────────────
EPISODE 2 SUMMARY
────────────────────────────────────────────────────────────────────────────────

Outcome:     ✓ SUCCESS
Distance:    918.3 m
Steps:       2000/2000
Avg Speed:   4.9 m/s (17.6 km/h)
Avg Lane:    0.067 m
Max Lane:    0.242 m

  Saving video (1260 frames)...
  ✓ Video: ppo_eval_videos/ppo_ep2_20260108_181554.mp4


================================================================================
OVERALL RESULTS
================================================================================

Success Rate:       100.0%
Avg Distance:       920.7 m
Avg Speed:          4.9 m/s (17.7 km/h)
Avg Lane Offset:    0.106 m

================================================================================
SCORING
================================================================================

Speed:          70.3%
Safety:         100.0%
Distance:       92.1%
Lane Keeping:   78.8%
OVERALL:        85.3%

GRADE: A - Excellent

VIDEOS:
  • ppo_eval_videos/ppo_ep1_20260108_181515.mp4
  • ppo_eval_videos/ppo_ep2_20260108_181554.mp4

================================================================================

================================================================================
PPO LANE KEEPING - EVALUATION
================================================================================
Model: /home/robotuser/carla_0.9.12/PythonAPI/examples/ppo_lane_keeping_final_continued.pth
Episodes: 1
Deterministic: True

carla_evaluation.py:3226: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy.load_state_dict(torch.load(model_path, map_location=device))
✓ Model loaded


================================================================================
EPISODE 1/1
================================================================================

  Step  200: Speed=5.1 m/s | Dist=467.0m | Lane=0.049m
  Step  400: Speed=5.1 m/s | Dist=517.6m | Lane=0.114m
  Step  600: Speed=5.1 m/s | Dist=568.2m | Lane=0.161m
  Step  800: Speed=5.1 m/s | Dist=618.8m | Lane=0.169m
  Step 1000: Speed=5.1 m/s | Dist=669.4m | Lane=0.136m
  Step 1200: Speed=5.1 m/s | Dist=720.0m | Lane=0.222m
  Step 1400: Speed=5.1 m/s | Dist=770.6m | Lane=0.171m
  Step 1600: Speed=5.1 m/s | Dist=821.2m | Lane=0.140m
  Step 1800: Speed=5.2 m/s | Dist=872.3m | Lane=0.425m
  Step 2000: Speed=5.1 m/s | Dist=923.1m | Lane=0.112m
  Step 2200: Speed=5.1 m/s | Dist=974.1m | Lane=0.237m
  Step 2400: Speed=5.1 m/s | Dist=1024.7m | Lane=0.006m

────────────────────────────────────────────────────────────────────────────────
EPISODE 1 SUMMARY
────────────────────────────────────────────────────────────────────────────────

Outcome:     ✓ SUCCESS
Distance:    1050.1 m
Steps:       2500/2500
Avg Speed:   5.0 m/s (17.9 km/h)
Avg Lane:    0.156 m
Max Lane:    1.351 m

  Saving video (1537 frames)...
  ✓ Video: ppo_eval_videos/ppo_ep1_20260108_183655.mp4


================================================================================
OVERALL RESULTS
================================================================================

Success Rate:       100.0%
Avg Distance:       1050.1 m
Avg Speed:          5.0 m/s (17.9 km/h)
Avg Lane Offset:    0.156 m

================================================================================
SCORING
================================================================================

Speed:          71.1%
Safety:         100.0%
Distance:       100.0%
Lane Keeping:   68.8%
OVERALL:        85.0%

GRADE: B - Good

VIDEOS:
  • ppo_eval_videos/ppo_ep1_20260108_183655.mp4

================================================================================